# 1. DBBE

In [1]:
# from sklearn.metrics import adjusted_rand_score, v_measure_score
# import matplotlib.pyplot as plt
# import seaborn as sns
# import re
# import pandas as pd
# import numpy as np
# from collections import Counter, defaultdict
# from typing import List, Dict, Tuple, Optional, Set
# import time
# from tqdm import tqdm
# import unicodedata

# try:
#     import cupy as cp
#     GPU_AVAILABLE = True
#     print("GPU detected - using CuPy acceleration")
# except ImportError:
#     cp = np
#     GPU_AVAILABLE = False
#     print("No GPU - using NumPy (CPU mode)")


# class TextPreprocessor:
#     def __init__(self, lowercase=True, remove_punctuation=True, remove_diacritics=True):
#         self.lowercase = lowercase
#         self.remove_punctuation = remove_punctuation
#         self.remove_diacritics = remove_diacritics
#         if remove_punctuation:
#             self.punct_pattern = re.compile(r'[^\w\s]', re.UNICODE)
#             self.remove_chars_pattern = re.compile(r'[\(\)\{\}]')

#     def _remove_diacritics(self, text: str) -> str:
#         return ''.join(
#             c for c in unicodedata.normalize('NFD', text)
#             if unicodedata.category(c) != 'Mn'
#         )

#     def preprocess(self, text: str) -> str:
#         if not isinstance(text, str):
#             text = str(text) if pd.notna(text) else ''

#         if self.remove_diacritics:
#             text = self._remove_diacritics(text)
#         if self.lowercase:
#             text = text.lower()
#         if self.remove_punctuation:
#             text = self.remove_chars_pattern.sub('', text)
#             text = self.punct_pattern.sub(' ', text)

#         return ' '.join(text.split())

#     def preprocess_batch(self, texts: List[str]) -> List[str]:
#         return [self.preprocess(t) for t in texts]


# class ShingleGenerator:
#     def __init__(self, shingle_size: int = 4, use_gpu: bool = GPU_AVAILABLE):
#         self.shingle_size = shingle_size
#         self.use_gpu = use_gpu and GPU_AVAILABLE
#         self.xp = cp if self.use_gpu else np

#     def generate_shingles(self, text: str) -> np.ndarray:
#         if len(text) < self.shingle_size:
#             return np.array([hash(text) % (2**31)], dtype=np.int32)

#         chars = self.xp.array([ord(c) for c in text], dtype=np.int32)
#         n_shingles = len(text) - self.shingle_size + 1

#         shingles = self.xp.zeros(n_shingles, dtype=np.int32)
#         for i in range(self.shingle_size):
#             shingles += chars[i:i+n_shingles] * (31 ** i)

#         unique_shingles = self.xp.unique(shingles)

#         if self.use_gpu:
#             unique_shingles = cp.asnumpy(unique_shingles)

#         return unique_shingles

#     def generate_batch(self, texts: List[str]) -> List[np.ndarray]:
#         return [self.generate_shingles(t) for t in texts]


# class MinHashProcessor:
#     def __init__(self, num_perm: int = 128, use_gpu: bool = GPU_AVAILABLE):
#         self.num_perm = num_perm
#         self.use_gpu = use_gpu and GPU_AVAILABLE
#         self.xp = cp if self.use_gpu else np

#         rng = self.xp.random.RandomState(42)
#         self.hash_a = rng.randint(1, 2**31-1, num_perm, dtype=np.int64)
#         self.hash_b = rng.randint(0, 2**31-1, num_perm, dtype=np.int64)
#         self.prime = np.int64(2**31-1)

#         if self.use_gpu:
#             print(f"Using GPU for MinHash ({num_perm} permutations)")

#     def compute_signature(self, shingles: np.ndarray) -> np.ndarray:
#         if len(shingles) == 0:
#             return np.full(self.num_perm, self.prime, dtype=np.int64)

#         if self.use_gpu:
#             shingles_gpu = self.xp.array(shingles, dtype=np.int64)
#         else:
#             shingles_gpu = shingles.astype(np.int64)

#         shingles_expanded = shingles_gpu[:, self.xp.newaxis]
#         hashes = (self.hash_a * shingles_expanded + self.hash_b) % self.prime
#         signature = self.xp.min(hashes, axis=0)

#         if self.use_gpu:
#             signature = cp.asnumpy(signature)

#         return signature

#     def compute_batch(self, shingles_batch: List[np.ndarray]) -> np.ndarray:
#         signatures = np.zeros((len(shingles_batch), self.num_perm), dtype=np.int64)
#         for i, shingles in enumerate(shingles_batch):
#             signatures[i] = self.compute_signature(shingles)
#         return signatures


# class LSHIndex:
#     def __init__(self, threshold: float = 0.3, num_perm: int = 128):
#         self.threshold = threshold
#         self.num_perm = num_perm
#         self.bands = 16
#         self.rows = num_perm // self.bands
#         self.signatures = []
#         self.num_docs = 0
#         self.hash_tables = [defaultdict(list) for _ in range(self.bands)]

#     def _hash_band(self, band: np.ndarray) -> int:
#         return int(hash(tuple(band)) % (2**31))

#     def insert_batch(self, signatures: np.ndarray, start_idx: int):
#         batch_size = signatures.shape[0]
#         self.signatures.append(signatures)

#         for band_idx in range(self.bands):
#             start_row = band_idx * self.rows
#             end_row = start_row + self.rows

#             for doc_idx in range(batch_size):
#                 band = signatures[doc_idx, start_row:end_row]
#                 band_hash = self._hash_band(band)
#                 global_doc_id = start_idx + doc_idx
#                 self.hash_tables[band_idx][band_hash].append(global_doc_id)

#         self.num_docs += batch_size

#     def query_batch(self, signatures: np.ndarray, start_idx: int) -> List[set]:
#         batch_size = signatures.shape[0]
#         candidates = [set() for _ in range(batch_size)]

#         for band_idx in range(self.bands):
#             start_row = band_idx * self.rows
#             end_row = start_row + self.rows

#             for doc_idx in range(batch_size):
#                 query_doc_id = start_idx + doc_idx
#                 band = signatures[doc_idx, start_row:end_row]
#                 band_hash = self._hash_band(band)
#                 bucket = self.hash_tables[band_idx].get(band_hash, [])
#                 candidates[doc_idx].update(c for c in bucket if c < query_doc_id)

#         return candidates


# class SimilarityComputer:
#     def __init__(self, threshold: float = 0.3, use_gpu: bool = GPU_AVAILABLE):
#         self.threshold = threshold
#         self.use_gpu = use_gpu and GPU_AVAILABLE
#         self.xp = cp if self.use_gpu else np

#     def compute_batch_similarities(self, query_sig: np.ndarray,
#                                    candidate_sigs: np.ndarray) -> np.ndarray:
#         if self.use_gpu:
#             query_gpu = self.xp.array(query_sig)
#             cands_gpu = self.xp.array(candidate_sigs)
#             query_expanded = self.xp.tile(query_gpu, (len(candidate_sigs), 1))
#             matches = self.xp.sum(query_expanded == cands_gpu, axis=1)
#             sims = matches.astype(np.float32) / query_sig.shape[0]
#             return cp.asnumpy(sims)
#         else:
#             query_expanded = np.tile(query_sig, (len(candidate_sigs), 1))
#             matches = np.sum(query_expanded == candidate_sigs, axis=1)
#             return matches.astype(np.float32) / query_sig.shape[0]


# class UnionFind:
#     def __init__(self, n: int):
#         self.parent = list(range(n))
#         self.rank = [0] * n

#     def find(self, x: int) -> int:
#         if self.parent[x] != x:
#             self.parent[x] = self.find(self.parent[x])
#         return self.parent[x]

#     def union(self, x: int, y: int):
#         px, py = self.find(x), self.find(y)
#         if px == py:
#             return
#         if self.rank[px] < self.rank[py]:
#             px, py = py, px
#         self.parent[py] = px
#         if self.rank[px] == self.rank[py]:
#             self.rank[px] += 1

#     def get_clusters(self) -> Dict[int, int]:
#         return {i: self.find(i) for i in range(len(self.parent))}


# class FastMinHashClustering:
#     def __init__(self, threshold: float = 0.3, shingle_size: int = 4,
#                  num_perm: int = 128, chunk_size: int = 50000,
#                  use_gpu: Optional[bool] = None):

#         if use_gpu is None:
#             use_gpu = GPU_AVAILABLE

#         self.threshold = threshold
#         self.chunk_size = chunk_size
#         self.use_gpu = use_gpu and GPU_AVAILABLE

#         self.preprocessor = TextPreprocessor(
#             lowercase=True,
#             remove_punctuation=True,
#             remove_diacritics=True
#         )
#         self.shingler = ShingleGenerator(shingle_size, use_gpu)
#         self.minhash = MinHashProcessor(num_perm, use_gpu)
#         self.lsh_index = LSHIndex(threshold, num_perm)
#         self.similarity_computer = SimilarityComputer(threshold, use_gpu)
#         self.all_similarities = []

#         mode = "GPU (CuPy)" if self.use_gpu else "CPU (NumPy)"
#         print(f"Initialized in {mode} mode")

#     def cluster(self, texts: List[str]) -> Tuple[Dict[int, int], List[Tuple[int, int, float]]]:
#         n_docs = len(texts)
#         n_chunks = (n_docs + self.chunk_size - 1) // self.chunk_size

#         print(f"\nClustering {n_docs:,} documents in {n_chunks} chunks")
#         print(f"threshold={self.threshold}, chunk_size={self.chunk_size:,}")

#         start_time = time.time()

#         for chunk_idx in tqdm(range(n_chunks), desc="Processing"):
#             chunk_start = chunk_idx * self.chunk_size
#             chunk_end = min(chunk_start + self.chunk_size, n_docs)
#             chunk_texts = texts[chunk_start:chunk_end]

#             processed = self.preprocessor.preprocess_batch(chunk_texts)
#             shingles = self.shingler.generate_batch(processed)
#             signatures = self.minhash.compute_batch(shingles)
#             self.lsh_index.insert_batch(signatures, chunk_start)

#             if chunk_start > 0:
#                 candidates = self.lsh_index.query_batch(signatures, chunk_start)

#                 for doc_idx, cand_set in enumerate(candidates):
#                     if not cand_set:
#                         continue

#                     query_doc_id = chunk_start + doc_idx
#                     query_sig = signatures[doc_idx]

#                     cand_list = sorted(cand_set)
#                     cand_sigs = []
#                     for cand_id in cand_list:
#                         batch_idx = cand_id // self.chunk_size
#                         local_idx = cand_id % self.chunk_size
#                         if batch_idx < len(self.lsh_index.signatures):
#                             cand_sigs.append(self.lsh_index.signatures[batch_idx][local_idx])

#                     if cand_sigs:
#                         cand_sigs = np.array(cand_sigs)
#                         sims = self.similarity_computer.compute_batch_similarities(
#                             query_sig, cand_sigs
#                         )

#                         for cand_id, sim in zip(cand_list[:len(sims)], sims):
#                             if sim >= self.threshold:
#                                 self.all_similarities.append((cand_id, query_doc_id, float(sim)))

#         elapsed = time.time() - start_time
#         print(f"\nFound {len(self.all_similarities):,} similarities in {elapsed:.2f}s")
#         print(f"Throughput: {n_docs/elapsed:,.0f} docs/sec")

#         print("Building clusters...")
#         uf = UnionFind(n_docs)
#         for doc1, doc2, _ in tqdm(self.all_similarities, desc="Clustering"):
#             uf.union(doc1, doc2)

#         clusters = uf.get_clusters()
#         n_clusters = len(set(clusters.values()))

#         total_time = time.time() - start_time
#         print(f"\nCreated {n_clusters:,} clusters in {total_time:.2f}s total")

#         return clusters, self.all_similarities


# def reconstruct_poems(df):
#     poem_to_clusters = defaultdict(set)
#     poem_verse_counts = defaultdict(int)

#     for _, row in df.iterrows():
#         poem_id = row['idoriginal_poem']
#         cluster_id = row['cluster_id']
#         poem_verse_counts[poem_id] += 1
#         if cluster_id != -1:
#             poem_to_clusters[poem_id].add(cluster_id)

#     print(f"\nReconstructed {len(poem_to_clusters)} poems")
#     return poem_to_clusters, poem_verse_counts


# def calculate_poem_cluster_similarity(clusters_a: Set[int], clusters_b: Set[int]) -> float:
#     if not clusters_a or not clusters_b:
#         return 0.0
#     intersection = len(clusters_a & clusters_b)
#     union = len(clusters_a | clusters_b)
#     return intersection / union if union > 0 else 0.0


# def cluster_poems(poem_to_clusters: Dict, similarity_threshold: float = 0.60):
#     poem_ids = list(poem_to_clusters.keys())
#     n_poems = len(poem_ids)

#     edges = []
#     for i in range(n_poems):
#         for j in range(i + 1, n_poems):
#             poem_a = poem_ids[i]
#             poem_b = poem_ids[j]
#             similarity = calculate_poem_cluster_similarity(
#                 poem_to_clusters[poem_a],
#                 poem_to_clusters[poem_b]
#             )
#             if similarity >= similarity_threshold:
#                 edges.append((poem_a, poem_b, similarity))

#     class PoemUnionFind:
#         def __init__(self, elements):
#             self.parent = {e: e for e in elements}
#             self.rank = {e: 0 for e in elements}

#         def find(self, x):
#             if self.parent[x] != x:
#                 self.parent[x] = self.find(self.parent[x])
#             return self.parent[x]

#         def union(self, x, y):
#             px, py = self.find(x), self.find(y)
#             if px == py:
#                 return
#             if self.rank[px] < self.rank[py]:
#                 px, py = py, px
#             self.parent[py] = px
#             if self.rank[px] == self.rank[py]:
#                 self.rank[px] += 1

#     uf = PoemUnionFind(poem_ids)
#     for poem_a, poem_b, _ in edges:
#         uf.union(poem_a, poem_b)

#     poem_clusters = {poem_id: uf.find(poem_id) for poem_id in poem_ids}
#     n_clusters = len(set(poem_clusters.values()))

#     return poem_clusters, edges, n_clusters


# def evaluate_clustering(y_true, y_pred):
#     ari = adjusted_rand_score(y_true, y_pred)
#     v_measure = v_measure_score(y_true, y_pred)
#     return ari, v_measure


# def calculate_perfect_reconstruction_rate(df, poem_clusters):
#     poem_to_type = df.groupby('idoriginal_poem')['type_id'].first().to_dict()

#     gt_to_poems = defaultdict(set)
#     for poem_id, gt_type in poem_to_type.items():
#         gt_to_poems[gt_type].add(poem_id)

#     pred_to_poems = defaultdict(set)
#     for poem_id, pred_cluster in poem_clusters.items():
#         pred_to_poems[pred_cluster].add(poem_id)

#     perfectly_reconstructed = 0
#     total_gt_clusters = len(gt_to_poems)

#     for gt_type, gt_poems in gt_to_poems.items():
#         for pred_cluster, pred_poems in pred_to_poems.items():
#             if gt_poems == pred_poems:
#                 perfectly_reconstructed += 1
#                 break

#     reconstruction_rate = perfectly_reconstructed / total_gt_clusters if total_gt_clusters > 0 else 0
#     return reconstruction_rate, perfectly_reconstructed, total_gt_clusters


# def visualize_verse_grid_search(results_df, save_path='verse_grid_search_results.png'):
#     # sns.set_palette("colorblind")

#     ari_pivot = results_df.pivot(index='shingle_size', columns='threshold', values='ari')
#     vmeasure_pivot = results_df.pivot(index='shingle_size', columns='threshold', values='v_measure')
#     clusters_pivot = results_df.pivot(index='shingle_size', columns='threshold', values='n_clusters')
#     similarities_pivot = results_df.pivot(index='shingle_size', columns='threshold', values='n_similarities')

#     fig, axes = plt.subplots(2, 2, figsize=(16, 12))
#     fig.suptitle('Verse-Level Clustering Grid Search Results', fontsize=18, fontweight='bold')

#     col_labels = [f"{col:.0%}" for col in ari_pivot.columns]

#     ax1 = axes[0, 0]
#     sns.heatmap(ari_pivot, annot=True, fmt='.4f', cmap='viridis', ax=ax1,
#                 cbar_kws={'label': 'ARI'}, xticklabels=col_labels)
#     ax1.set_xlabel('Similarity Threshold', fontweight='bold', fontsize=12)
#     ax1.set_ylabel('Shingle Size', fontweight='bold', fontsize=12)
#     ax1.set_title('Adjusted Rand Index (ARI)', fontweight='bold', fontsize=13)

#     ax2 = axes[0, 1]
#     sns.heatmap(vmeasure_pivot, annot=True, fmt='.4f', cmap='viridis', ax=ax2,
#                 cbar_kws={'label': 'V-measure'}, xticklabels=col_labels)
#     ax2.set_xlabel('Similarity Threshold', fontweight='bold', fontsize=12)
#     ax2.set_ylabel('Shingle Size', fontweight='bold', fontsize=12)
#     ax2.set_title('V-measure', fontweight='bold', fontsize=13)

#     ax3 = axes[1, 0]
#     sns.heatmap(clusters_pivot, annot=True, fmt='.0f', cmap='viridis', ax=ax3,
#                 cbar_kws={'label': 'Clusters'}, xticklabels=col_labels)
#     ax3.set_xlabel('Similarity Threshold', fontweight='bold', fontsize=12)
#     ax3.set_ylabel('Shingle Size', fontweight='bold', fontsize=12)
#     ax3.set_title('Number of Clusters', fontweight='bold', fontsize=13)

#     ax4 = axes[1, 1]
#     sns.heatmap(similarities_pivot, annot=True, fmt='.0f', cmap='viridis', ax=ax4,
#                 cbar_kws={'label': 'Similarities'}, xticklabels=col_labels)
#     ax4.set_xlabel('Similarity Threshold', fontweight='bold', fontsize=12)
#     ax4.set_ylabel('Shingle Size', fontweight='bold', fontsize=12)
#     ax4.set_title('Number of Similarities Found', fontweight='bold', fontsize=13)

#     plt.tight_layout()
#     plt.savefig(save_path, dpi=300, bbox_inches='tight')
#     print(f"\nVisualization saved to: {save_path}")
#     plt.close()

# import matplotlib.cm as cm
# import matplotlib.pyplot as plt
# import matplotlib.cm as cm
# import numpy as np

# def visualize_poem_grid_search(results_df, save_path='poem_grid_search_results.png'):
#     fig, axes = plt.subplots(2, 2, figsize=(14, 10))
#     fig.suptitle('Poem-Level Clustering Grid Search Results', fontsize=16, fontweight='bold')

#     thresholds = results_df['threshold'].values
#     thresholds_pct = [f"{t:.0%}" for t in thresholds]

#     # Normalize values for colormap
#     def normalize(vals):
#         return (vals - np.min(vals)) / (np.max(vals) - np.min(vals))

#     # ---------- AX1: ARI ----------
#     ax1 = axes[0, 0]
#     norm_vals = normalize(results_df['ari'].values)
#     colors = cm.viridis(norm_vals)
#     ax1.plot(thresholds_pct, results_df['ari'].values, marker='o', linewidth=2, markersize=8)
#     for i, (x, y) in enumerate(zip(thresholds_pct, results_df['ari'].values)):
#         # ax1.plot(x, y, marker='o', color=colors[i], markersize=10)
#         ax1.text(i, y, f'{y:.4f}', ha='center', va='bottom', fontsize=9)
#     ax1.set_xlabel('Similarity Threshold', fontweight='bold')
#     ax1.set_ylabel('Adjusted Rand Index (ARI)', fontweight='bold')
#     ax1.set_title('ARI vs Threshold')
#     ax1.grid(True, alpha=0.3)

#     # ---------- AX2: V-measure ----------
#     ax2 = axes[0, 1]
#     norm_vals = normalize(results_df['v_measure'].values)
#     colors = cm.viridis(norm_vals)
#     ax2.plot(thresholds_pct, results_df['v_measure'].values, marker='o', linewidth=2, markersize=8)
#     for i, (x, y) in enumerate(zip(thresholds_pct, results_df['v_measure'].values)):
#         # ax2.plot(x, y, marker='o', color=colors[i], markersize=10)
#         ax2.text(i, y, f'{y:.4f}', ha='center', va='bottom', fontsize=9)
#     ax2.set_xlabel('Similarity Threshold', fontweight='bold')
#     ax2.set_ylabel('V-measure', fontweight='bold')
#     ax2.set_title('V-measure vs Threshold')
#     ax2.grid(True, alpha=0.3)

#     # ---------- AX3: Perfect Reconstruction Rate ----------
#     ax3 = axes[1, 0]
#     prr_vals = results_df['perfect_reconstruction_rate'].values * 100
#     norm_vals = normalize(prr_vals)
#     colors = cm.viridis(norm_vals)
#     ax3.plot(thresholds_pct, prr_vals, marker='o', linewidth=2, markersize=8)
#     for i, (x, y) in enumerate(zip(thresholds_pct, prr_vals)):
#         ax3.plot(x, y, marker='o', color=colors[i], markersize=10)
#         ax3.text(i, y, f'{y:.1f}%', ha='center', va='bottom', fontsize=9)
#     ax3.set_xlabel('Similarity Threshold', fontweight='bold')
#     ax3.set_ylabel('Perfect Reconstruction Rate (%)', fontweight='bold')
#     ax3.set_title('Perfect Reconstruction Rate vs Threshold')
#     ax3.grid(True, alpha=0.3)

#     # ---------- AX4: Number of Clusters ----------
#     ax4 = axes[1, 1]
#     n_clusters_vals = results_df['n_clusters'].values
#     norm_vals = normalize(n_clusters_vals)
#     colors = cm.viridis(norm_vals)
#     ax4.plot(thresholds_pct, n_clusters_vals, marker='o', linewidth=2, markersize=8)
#     for i, (x, y) in enumerate(zip(thresholds_pct, n_clusters_vals)):
#         ax4.plot(x, y, marker='o', color=colors[i], markersize=10)
#         ax4.text(i, y, f'{y}', ha='center', va='bottom', fontsize=9)
#     ax4.set_xlabel('Similarity Threshold', fontweight='bold')
#     ax4.set_ylabel('Number of Poem Clusters', fontweight='bold')
#     ax4.set_title('Number of Clusters vs Threshold')
#     ax4.grid(True, alpha=0.3)

#     plt.tight_layout()
#     plt.savefig(save_path, dpi=300, bbox_inches='tight')
#     print(f"\nVisualization saved to: {save_path}")
#     plt.close()



# def verse_level_grid_search(texts, df, thresholds, shingle_sizes, num_perm=128):
#     results = []
#     best_ari = -1
#     best_threshold = None
#     best_shingle_size = None
#     best_clusters = None
#     best_similarities = None

#     print("\n" + "="*100)
#     print("VERSE-LEVEL 2D GRID SEARCH")
#     print("="*100)
#     print(f"\nTesting thresholds: {[f'{t:.0%}' for t in thresholds]}")
#     print(f"Testing shingle sizes: {shingle_sizes}\n")

#     total_combinations = len(thresholds) * len(shingle_sizes)
#     print(f"Total combinations: {total_combinations}\n")

#     for shingle_size in shingle_sizes:
#         for threshold in thresholds:
#             print(f"\nTesting shingle_size={shingle_size}, threshold={threshold:.0%}...")

#             clusterer = FastMinHashClustering(
#                 threshold=threshold,
#                 shingle_size=shingle_size,
#                 num_perm=num_perm,
#                 chunk_size=1
#             )

#             clusters, similarities = clusterer.cluster(texts)

#             if 'idgroup' in df.columns:
#                 temp_df = df.copy()
#                 temp_df['cluster_id'] = temp_df.index.map(clusters)

#                 mask = temp_df['idgroup'].notna() & temp_df['cluster_id'].notna()
#                 y_true = temp_df.loc[mask, 'idgroup'].tolist()
#                 y_pred = temp_df.loc[mask, 'cluster_id'].tolist()

#                 ari, v_measure = evaluate_clustering(y_true, y_pred)
#                 n_gt_clusters = len(set(y_true))
#             else:
#                 ari, v_measure = 0, 0
#                 n_gt_clusters = 0

#             n_clusters = len(set(clusters.values()))

#             results.append({
#                 'shingle_size': shingle_size,
#                 'threshold': threshold,
#                 'n_clusters': n_clusters,
#                 'n_similarities': len(similarities),
#                 'ari': ari,
#                 'v_measure': v_measure,
#                 'n_gt_clusters': n_gt_clusters
#             })

#             if ari > best_ari:
#                 best_ari = ari
#                 best_threshold = threshold
#                 best_shingle_size = shingle_size
#                 best_clusters = clusters
#                 best_similarities = similarities

#             print(f"  ARI: {ari:.4f}, V-measure: {v_measure:.4f}, Clusters: {n_clusters}")

#     results_df = pd.DataFrame(results)

#     print("\n" + "="*100)
#     print("VERSE-LEVEL GRID SEARCH SUMMARY")
#     print("="*100)
#     print(f"\n{'Shingle':<10} {'Threshold':<12} {'Clusters':<10} {'Similarities':<15} {'ARI':<8} {'V-measure':<12}")
#     print("-" * 80)

#     for _, result in results_df.iterrows():
#         print(f"{result['shingle_size']:<10} "
#               f"{result['threshold']:<12.0%} "
#               f"{result['n_clusters']:<10} "
#               f"{result['n_similarities']:<15} "
#               f"{result['ari']:<8.4f} "
#               f"{result['v_measure']:<12.4f}")

#     print(f"\n{'='*100}")
#     print("BEST VERSE-LEVEL PARAMETERS")
#     print("="*100)
#     print(f"\nBest parameters by ARI:")
#     print(f"  Shingle size: {best_shingle_size}")
#     print(f"  Threshold: {best_threshold:.0%}")
#     best_result = results_df[(results_df['threshold'] == best_threshold) &
#                               (results_df['shingle_size'] == best_shingle_size)].iloc[0]
#     print(f"  ARI: {best_result['ari']:.4f}")
#     print(f"  V-measure: {best_result['v_measure']:.4f}")
#     print(f"  Number of clusters: {best_result['n_clusters']}")
#     print(f"  Number of similarities found: {best_result['n_similarities']}")

#     visualize_verse_grid_search(results_df)

#     return best_clusters, best_similarities, best_threshold, best_shingle_size, results_df


# def poem_level_grid_search(df, poem_to_clusters, thresholds):
#     results = []
#     best_ari = -1
#     best_threshold = None
#     best_poem_clusters = None

#     print("\n" + "="*100)
#     print("POEM-LEVEL GRID SEARCH")
#     print("="*100)
#     print(f"\nTesting thresholds: {[f'{t:.0%}' for t in thresholds]}\n")

#     for threshold in thresholds:
#         print(f"\nTesting threshold {threshold:.0%}...")

#         poem_clusters, poem_edges, n_clusters = cluster_poems(poem_to_clusters, threshold)

#         if 'type_id' in df.columns:
#             poem_to_type = df.groupby('idoriginal_poem')['type_id'].first().to_dict()

#             y_true = []
#             y_pred = []
#             for poem_id, predicted_cluster in poem_clusters.items():
#                 if poem_id in poem_to_type:
#                     y_true.append(poem_to_type[poem_id])
#                     y_pred.append(predicted_cluster)

#             ari, v_measure = evaluate_clustering(y_true, y_pred)
#             reconstruction_rate, n_perfect, n_total_gt = calculate_perfect_reconstruction_rate(df, poem_clusters)
#         else:
#             ari, v_measure = 0, 0
#             reconstruction_rate, n_perfect, n_total_gt = 0, 0, 0

#         results.append({
#             'threshold': threshold,
#             'n_clusters': n_clusters,
#             'n_edges': len(poem_edges),
#             'ari': ari,
#             'v_measure': v_measure,
#             'perfect_reconstruction_rate': reconstruction_rate,
#             'n_perfect_clusters': n_perfect,
#             'n_total_gt_clusters': n_total_gt
#         })

#         if ari > best_ari:
#             best_ari = ari
#             best_threshold = threshold
#             best_poem_clusters = poem_clusters

#         print(f"  ARI: {ari:.4f}, V-measure: {v_measure:.4f}, Clusters: {n_clusters}")

#     results_df = pd.DataFrame(results)

#     print("\n" + "="*100)
#     print("POEM-LEVEL GRID SEARCH SUMMARY")
#     print("="*100)
#     print(f"\n{'Threshold':<12} {'Clusters':<10} {'Edges':<10} {'ARI':<8} {'V-measure':<12} {'Perfect Recon.':<15}")
#     print("-" * 80)

#     for _, result in results_df.iterrows():
#         print(f"{result['threshold']:<12.0%} "
#               f"{result['n_clusters']:<10} "
#               f"{result['n_edges']:<10} "
#               f"{result['ari']:<8.4f} "
#               f"{result['v_measure']:<12.4f} "
#               f"{result['perfect_reconstruction_rate']:<15.1%}")

#     print(f"\n{'='*100}")
#     print("BEST POEM-LEVEL THRESHOLD")
#     print("="*100)
#     print(f"\nBest threshold by ARI: {best_threshold:.0%}")
#     best_result = results_df[results_df['threshold'] == best_threshold].iloc[0]
#     print(f"  ARI: {best_result['ari']:.4f}")
#     print(f"  V-measure: {best_result['v_measure']:.4f}")
#     print(f"  Perfect reconstruction rate: {best_result['perfect_reconstruction_rate']:.1%}")
#     print(f"    ({best_result['n_perfect_clusters']:.0f}/{best_result['n_total_gt_clusters']:.0f} GT clusters perfectly reconstructed)")

#     visualize_poem_grid_search(results_df)

#     return best_poem_clusters, best_threshold, results_df


# def main():
#     DATA_FILE = 'dbbe_full.csv'

#     print("="*100)
#     print("LOADING DATA")
#     print("="*100)

#     df = pd.read_csv(DATA_FILE)

#     if 'verse' in df.columns:
#         df['text'] = df['verse']
#     elif 'text' not in df.columns:
#         raise ValueError("Dataset must have either 'verse' or 'text' column")

#     df['text'] = df['text'].fillna('').astype(str)
#     print(f"\nLoaded {df.shape[0]:,} verses")

#     texts = df['text'].tolist()

#     verse_thresholds = [0.2, 0.3, 0.4, 0.5]
#     shingle_sizes = [2, 3, 4, 5]

#     best_clusters, best_similarities, best_verse_threshold, best_shingle_size, verse_results = verse_level_grid_search(
#         texts, df, verse_thresholds, shingle_sizes, num_perm=128
#     )

#     df['cluster_id'] = df.index.map(best_clusters)

#     sim_dict = defaultdict(list)
#     for doc1, doc2, sim in best_similarities:
#         sim_dict[doc1].append(sim)
#         sim_dict[doc2].append(sim)

#     df['certainty'] = df.index.map(
#         lambda i: np.mean(sim_dict[i]) if i in sim_dict else 1.0
#     )

#     preprocessor = TextPreprocessor(lowercase=True, remove_punctuation=True, remove_diacritics=True)
#     df['text_preprocessed'] = df['text'].apply(preprocessor.preprocess)

#     verse_output = "dbbe_verse_clustered_results.csv"
#     df.to_csv(verse_output, index=False)
#     print(f"\n{verse_output} saved with best parameters (shingle_size={best_shingle_size}, threshold={best_verse_threshold:.0%})")

#     if 'idoriginal_poem' in df.columns and 'type_id' in df.columns:
#         poem_to_clusters, poem_verse_counts = reconstruct_poems(df)

#         poem_thresholds = [0.50, 0.60, 0.70, 0.8]

#         best_poem_clusters, best_poem_threshold, poem_results = poem_level_grid_search(
#             df, poem_to_clusters, poem_thresholds
#         )

#         df['poem_cluster_id'] = df['idoriginal_poem'].map(best_poem_clusters)

#         poem_output = "dbbe_poem_level_clusters.csv"
#         df.to_csv(poem_output, index=False)
#         print(f"\n{poem_output} saved with best threshold ({best_poem_threshold:.0%})")
#     else:
#         print("\n" + "="*100)
#         print("SKIPPING POEM-LEVEL CLUSTERING")
#         print("="*100)
#         print("\nRequired columns 'idoriginal_poem' and/or 'type_id' not found")

#     print("\n" + "="*100)
#     print("ANALYSIS COMPLETE")
#     print("="*100)


# if __name__ == "__main__":
#     main()

from sklearn.metrics import adjusted_rand_score, v_measure_score
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
from typing import List, Dict, Tuple, Optional, Set
import time
from tqdm import tqdm
import unicodedata
from pathlib import Path

RESULTS_DIR = Path("dbbe_orthographic_results")
RESULTS_DIR.mkdir(exist_ok=True)

try:
    import cupy as cp
    GPU_AVAILABLE = True
    print("GPU detected - using CuPy acceleration")
except ImportError:
    cp = np
    GPU_AVAILABLE = False
    print("No GPU - using NumPy (CPU mode)")


class TextPreprocessor:
    def __init__(self, lowercase=True, remove_punctuation=True, remove_diacritics=True):
        self.lowercase = lowercase
        self.remove_punctuation = remove_punctuation
        self.remove_diacritics = remove_diacritics
        if remove_punctuation:
            self.punct_pattern = re.compile(r'[^\w\s]', re.UNICODE)
            self.remove_chars_pattern = re.compile(r'[\(\)\{\}]')

    def _remove_diacritics(self, text: str) -> str:
        return ''.join(
            c for c in unicodedata.normalize('NFD', text)
            if unicodedata.category(c) != 'Mn'
        )

    def preprocess(self, text: str) -> str:
        if not isinstance(text, str):
            text = str(text) if pd.notna(text) else ''

        if self.remove_diacritics:
            text = self._remove_diacritics(text)
        if self.lowercase:
            text = text.lower()
        if self.remove_punctuation:
            text = self.remove_chars_pattern.sub('', text)
            text = self.punct_pattern.sub(' ', text)

        return ' '.join(text.split())

    def preprocess_batch(self, texts: List[str]) -> List[str]:
        return [self.preprocess(t) for t in texts]


class ShingleGenerator:
    def __init__(self, shingle_size: int = 4, use_gpu: bool = GPU_AVAILABLE):
        self.shingle_size = shingle_size
        self.use_gpu = use_gpu and GPU_AVAILABLE
        self.xp = cp if self.use_gpu else np

    def generate_shingles(self, text: str) -> np.ndarray:
        if len(text) < self.shingle_size:
            return np.array([hash(text) % (2**31)], dtype=np.int32)

        chars = self.xp.array([ord(c) for c in text], dtype=np.int32)
        n_shingles = len(text) - self.shingle_size + 1

        shingles = self.xp.zeros(n_shingles, dtype=np.int32)
        for i in range(self.shingle_size):
            shingles += chars[i:i+n_shingles] * (31 ** i)

        unique_shingles = self.xp.unique(shingles)

        if self.use_gpu:
            unique_shingles = cp.asnumpy(unique_shingles)

        return unique_shingles

    def generate_batch(self, texts: List[str]) -> List[np.ndarray]:
        return [self.generate_shingles(t) for t in texts]


class MinHashProcessor:
    def __init__(self, num_perm: int = 128, use_gpu: bool = GPU_AVAILABLE):
        self.num_perm = num_perm
        self.use_gpu = use_gpu and GPU_AVAILABLE
        self.xp = cp if self.use_gpu else np

        rng = self.xp.random.RandomState(42)
        self.hash_a = rng.randint(1, 2**31-1, num_perm, dtype=np.int64)
        self.hash_b = rng.randint(0, 2**31-1, num_perm, dtype=np.int64)
        self.prime = np.int64(2**31-1)

        if self.use_gpu:
            print(f"Using GPU for MinHash ({num_perm} permutations)")

    def compute_signature(self, shingles: np.ndarray) -> np.ndarray:
        if len(shingles) == 0:
            return np.full(self.num_perm, self.prime, dtype=np.int64)

        if self.use_gpu:
            shingles_gpu = self.xp.array(shingles, dtype=np.int64)
        else:
            shingles_gpu = shingles.astype(np.int64)

        shingles_expanded = shingles_gpu[:, self.xp.newaxis]
        hashes = (self.hash_a * shingles_expanded + self.hash_b) % self.prime
        signature = self.xp.min(hashes, axis=0)

        if self.use_gpu:
            signature = cp.asnumpy(signature)

        return signature

    def compute_batch(self, shingles_batch: List[np.ndarray]) -> np.ndarray:
        signatures = np.zeros((len(shingles_batch), self.num_perm), dtype=np.int64)
        for i, shingles in enumerate(shingles_batch):
            signatures[i] = self.compute_signature(shingles)
        return signatures


class LSHIndex:
    def __init__(self, threshold: float = 0.3, num_perm: int = 128):
        self.threshold = threshold
        self.num_perm = num_perm
        self.bands = 16
        self.rows = num_perm // self.bands
        self.signatures = []
        self.num_docs = 0
        self.hash_tables = [defaultdict(list) for _ in range(self.bands)]

    def _hash_band(self, band: np.ndarray) -> int:
        return int(hash(tuple(band)) % (2**31))

    def insert_batch(self, signatures: np.ndarray, start_idx: int):
        batch_size = signatures.shape[0]
        self.signatures.append(signatures)

        for band_idx in range(self.bands):
            start_row = band_idx * self.rows
            end_row = start_row + self.rows

            for doc_idx in range(batch_size):
                band = signatures[doc_idx, start_row:end_row]
                band_hash = self._hash_band(band)
                global_doc_id = start_idx + doc_idx
                self.hash_tables[band_idx][band_hash].append(global_doc_id)

        self.num_docs += batch_size

    def query_batch(self, signatures: np.ndarray, start_idx: int) -> List[set]:
        batch_size = signatures.shape[0]
        candidates = [set() for _ in range(batch_size)]

        for band_idx in range(self.bands):
            start_row = band_idx * self.rows
            end_row = start_row + self.rows

            for doc_idx in range(batch_size):
                query_doc_id = start_idx + doc_idx
                band = signatures[doc_idx, start_row:end_row]
                band_hash = self._hash_band(band)
                bucket = self.hash_tables[band_idx].get(band_hash, [])
                candidates[doc_idx].update(c for c in bucket if c < query_doc_id)

        return candidates


class SimilarityComputer:
    def __init__(self, threshold: float = 0.3, use_gpu: bool = GPU_AVAILABLE):
        self.threshold = threshold
        self.use_gpu = use_gpu and GPU_AVAILABLE
        self.xp = cp if self.use_gpu else np

    def compute_batch_similarities(self, query_sig: np.ndarray,
                                   candidate_sigs: np.ndarray) -> np.ndarray:
        if self.use_gpu:
            query_gpu = self.xp.array(query_sig)
            cands_gpu = self.xp.array(candidate_sigs)
            query_expanded = self.xp.tile(query_gpu, (len(candidate_sigs), 1))
            matches = self.xp.sum(query_expanded == cands_gpu, axis=1)
            sims = matches.astype(np.float32) / query_sig.shape[0]
            return cp.asnumpy(sims)
        else:
            query_expanded = np.tile(query_sig, (len(candidate_sigs), 1))
            matches = np.sum(query_expanded == candidate_sigs, axis=1)
            return matches.astype(np.float32) / query_sig.shape[0]


class UnionFind:
    def __init__(self, n: int):
        self.parent = list(range(n))
        self.rank = [0] * n

    def find(self, x: int) -> int:
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x: int, y: int):
        px, py = self.find(x), self.find(y)
        if px == py:
            return
        if self.rank[px] < self.rank[py]:
            px, py = py, px
        self.parent[py] = px
        if self.rank[px] == self.rank[py]:
            self.rank[px] += 1

    def get_clusters(self) -> Dict[int, int]:
        return {i: self.find(i) for i in range(len(self.parent))}


class FastMinHashClustering:
    def __init__(self, threshold: float = 0.3, shingle_size: int = 4,
                 num_perm: int = 128, chunk_size: int = 50000,
                 use_gpu: Optional[bool] = None):

        if use_gpu is None:
            use_gpu = GPU_AVAILABLE

        self.threshold = threshold
        self.chunk_size = chunk_size
        self.use_gpu = use_gpu and GPU_AVAILABLE

        self.preprocessor = TextPreprocessor(
            lowercase=True,
            remove_punctuation=True,
            remove_diacritics=True
        )
        self.shingler = ShingleGenerator(shingle_size, use_gpu)
        self.minhash = MinHashProcessor(num_perm, use_gpu)
        self.lsh_index = LSHIndex(threshold, num_perm)
        self.similarity_computer = SimilarityComputer(threshold, use_gpu)
        self.all_similarities = []

        mode = "GPU (CuPy)" if self.use_gpu else "CPU (NumPy)"
        print(f"Initialized in {mode} mode")

    def cluster(self, texts: List[str]) -> Tuple[Dict[int, int], List[Tuple[int, int, float]]]:
        n_docs = len(texts)
        n_chunks = (n_docs + self.chunk_size - 1) // self.chunk_size

        print(f"\nClustering {n_docs:,} documents in {n_chunks} chunks")
        print(f"threshold={self.threshold}, chunk_size={self.chunk_size:,}")

        start_time = time.time()

        for chunk_idx in tqdm(range(n_chunks), desc="Processing"):
            chunk_start = chunk_idx * self.chunk_size
            chunk_end = min(chunk_start + self.chunk_size, n_docs)
            chunk_texts = texts[chunk_start:chunk_end]

            processed = self.preprocessor.preprocess_batch(chunk_texts)
            shingles = self.shingler.generate_batch(processed)
            signatures = self.minhash.compute_batch(shingles)
            self.lsh_index.insert_batch(signatures, chunk_start)

            if chunk_start > 0:
                candidates = self.lsh_index.query_batch(signatures, chunk_start)

                for doc_idx, cand_set in enumerate(candidates):
                    if not cand_set:
                        continue

                    query_doc_id = chunk_start + doc_idx
                    query_sig = signatures[doc_idx]

                    cand_list = sorted(cand_set)
                    cand_sigs = []
                    for cand_id in cand_list:
                        batch_idx = cand_id // self.chunk_size
                        local_idx = cand_id % self.chunk_size
                        if batch_idx < len(self.lsh_index.signatures):
                            cand_sigs.append(self.lsh_index.signatures[batch_idx][local_idx])

                    if cand_sigs:
                        cand_sigs = np.array(cand_sigs)
                        sims = self.similarity_computer.compute_batch_similarities(
                            query_sig, cand_sigs
                        )

                        for cand_id, sim in zip(cand_list[:len(sims)], sims):
                            if sim >= self.threshold:
                                self.all_similarities.append((cand_id, query_doc_id, float(sim)))

        elapsed = time.time() - start_time
        print(f"\nFound {len(self.all_similarities):,} similarities in {elapsed:.2f}s")
        print(f"Throughput: {n_docs/elapsed:,.0f} docs/sec")

        print("Building clusters...")
        uf = UnionFind(n_docs)
        for doc1, doc2, _ in tqdm(self.all_similarities, desc="Clustering"):
            uf.union(doc1, doc2)

        clusters = uf.get_clusters()
        n_clusters = len(set(clusters.values()))

        total_time = time.time() - start_time
        print(f"\nCreated {n_clusters:,} clusters in {total_time:.2f}s total")

        return clusters, self.all_similarities


def reconstruct_poems(df):
    poem_to_clusters = defaultdict(set)
    poem_verse_counts = defaultdict(int)

    for _, row in df.iterrows():
        poem_id = row['idoriginal_poem']
        cluster_id = row['cluster_id']
        poem_verse_counts[poem_id] += 1
        if cluster_id != -1:
            poem_to_clusters[poem_id].add(cluster_id)

    print(f"\nReconstructed {len(poem_to_clusters)} poems")
    return poem_to_clusters, poem_verse_counts


def calculate_poem_cluster_similarity(clusters_a: Set[int], clusters_b: Set[int]) -> float:
    if not clusters_a or not clusters_b:
        return 0.0
    intersection = len(clusters_a & clusters_b)
    union = len(clusters_a | clusters_b)
    return intersection / union if union > 0 else 0.0


def cluster_poems(poem_to_clusters: Dict, similarity_threshold: float = 0.60):
    poem_ids = list(poem_to_clusters.keys())
    n_poems = len(poem_ids)

    edges = []
    for i in range(n_poems):
        for j in range(i + 1, n_poems):
            poem_a = poem_ids[i]
            poem_b = poem_ids[j]
            similarity = calculate_poem_cluster_similarity(
                poem_to_clusters[poem_a],
                poem_to_clusters[poem_b]
            )
            if similarity >= similarity_threshold:
                edges.append((poem_a, poem_b, similarity))

    class PoemUnionFind:
        def __init__(self, elements):
            self.parent = {e: e for e in elements}
            self.rank = {e: 0 for e in elements}

        def find(self, x):
            if self.parent[x] != x:
                self.parent[x] = self.find(self.parent[x])
            return self.parent[x]

        def union(self, x, y):
            px, py = self.find(x), self.find(y)
            if px == py:
                return
            if self.rank[px] < self.rank[py]:
                px, py = py, px
            self.parent[py] = px
            if self.rank[px] == self.rank[py]:
                self.rank[px] += 1

    uf = PoemUnionFind(poem_ids)
    for poem_a, poem_b, _ in edges:
        uf.union(poem_a, poem_b)

    poem_clusters = {poem_id: uf.find(poem_id) for poem_id in poem_ids}
    n_clusters = len(set(poem_clusters.values()))

    return poem_clusters, edges, n_clusters


def evaluate_clustering(y_true, y_pred):
    ari = adjusted_rand_score(y_true, y_pred)
    v_measure = v_measure_score(y_true, y_pred)
    return ari, v_measure


def calculate_perfect_reconstruction_rate(df, poem_clusters):
    poem_to_type = df.groupby('idoriginal_poem')['type_id'].first().to_dict()

    gt_to_poems = defaultdict(set)
    for poem_id, gt_type in poem_to_type.items():
        gt_to_poems[gt_type].add(poem_id)

    pred_to_poems = defaultdict(set)
    for poem_id, pred_cluster in poem_clusters.items():
        pred_to_poems[pred_cluster].add(poem_id)

    perfectly_reconstructed = 0
    total_gt_clusters = len(gt_to_poems)

    for gt_type, gt_poems in gt_to_poems.items():
        for pred_cluster, pred_poems in pred_to_poems.items():
            if gt_poems == pred_poems:
                perfectly_reconstructed += 1
                break

    reconstruction_rate = perfectly_reconstructed / total_gt_clusters if total_gt_clusters > 0 else 0
    return reconstruction_rate, perfectly_reconstructed, total_gt_clusters


def visualize_verse_grid_search(results_df, save_path=None):
    if save_path is None:
        save_path = RESULTS_DIR / 'verse_grid_search_results.png'

    ari_pivot = results_df.pivot(index='shingle_size', columns='threshold', values='ari')
    vmeasure_pivot = results_df.pivot(index='shingle_size', columns='threshold', values='v_measure')
    clusters_pivot = results_df.pivot(index='shingle_size', columns='threshold', values='n_clusters')
    similarities_pivot = results_df.pivot(index='shingle_size', columns='threshold', values='n_similarities')

    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Verse-Level Clustering Grid Search Results', fontsize=18, fontweight='bold')

    col_labels = [f"{col:.0%}" for col in ari_pivot.columns]

    ax1 = axes[0, 0]
    sns.heatmap(ari_pivot, annot=True, fmt='.4f', cmap='viridis', ax=ax1,
                cbar_kws={'label': 'ARI'}, xticklabels=col_labels)
    ax1.set_xlabel('Similarity Threshold', fontweight='bold', fontsize=12)
    ax1.set_ylabel('Shingle Size', fontweight='bold', fontsize=12)
    ax1.set_title('Adjusted Rand Index (ARI)', fontweight='bold', fontsize=13)

    ax2 = axes[0, 1]
    sns.heatmap(vmeasure_pivot, annot=True, fmt='.4f', cmap='viridis', ax=ax2,
                cbar_kws={'label': 'V-measure'}, xticklabels=col_labels)
    ax2.set_xlabel('Similarity Threshold', fontweight='bold', fontsize=12)
    ax2.set_ylabel('Shingle Size', fontweight='bold', fontsize=12)
    ax2.set_title('V-measure', fontweight='bold', fontsize=13)

    ax3 = axes[1, 0]
    sns.heatmap(clusters_pivot, annot=True, fmt='.0f', cmap='viridis', ax=ax3,
                cbar_kws={'label': 'Clusters'}, xticklabels=col_labels)
    ax3.set_xlabel('Similarity Threshold', fontweight='bold', fontsize=12)
    ax3.set_ylabel('Shingle Size', fontweight='bold', fontsize=12)
    ax3.set_title('Number of Clusters', fontweight='bold', fontsize=13)

    ax4 = axes[1, 1]
    sns.heatmap(similarities_pivot, annot=True, fmt='.0f', cmap='viridis', ax=ax4,
                cbar_kws={'label': 'Similarities'}, xticklabels=col_labels)
    ax4.set_xlabel('Similarity Threshold', fontweight='bold', fontsize=12)
    ax4.set_ylabel('Shingle Size', fontweight='bold', fontsize=12)
    ax4.set_title('Number of Similarities Found', fontweight='bold', fontsize=13)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"\nVisualization saved to: {save_path}")
    plt.close()


def visualize_poem_grid_search(results_df, save_path=None):
    if save_path is None:
        save_path = RESULTS_DIR / 'poem_grid_search_results.png'
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Poem-Level Clustering Grid Search Results', fontsize=16, fontweight='bold')

    thresholds = results_df['threshold'].values
    thresholds_pct = [f"{t:.0%}" for t in thresholds]

    def normalize(vals):
        return (vals - np.min(vals)) / (np.max(vals) - np.min(vals))

    ax1 = axes[0, 0]
    norm_vals = normalize(results_df['ari'].values)
    colors = plt.cm.viridis(norm_vals)
    ax1.plot(thresholds_pct, results_df['ari'].values, marker='o', linewidth=2, markersize=8)
    for i, (x, y) in enumerate(zip(thresholds_pct, results_df['ari'].values)):
        ax1.text(i, y, f'{y:.4f}', ha='center', va='bottom', fontsize=9)
    ax1.set_xlabel('Similarity Threshold', fontweight='bold')
    ax1.set_ylabel('Adjusted Rand Index (ARI)', fontweight='bold')
    ax1.set_title('ARI vs Threshold')
    ax1.grid(True, alpha=0.3)

    ax2 = axes[0, 1]
    norm_vals = normalize(results_df['v_measure'].values)
    colors = plt.cm.viridis(norm_vals)
    ax2.plot(thresholds_pct, results_df['v_measure'].values, marker='o', linewidth=2, markersize=8)
    for i, (x, y) in enumerate(zip(thresholds_pct, results_df['v_measure'].values)):
        ax2.text(i, y, f'{y:.4f}', ha='center', va='bottom', fontsize=9)
    ax2.set_xlabel('Similarity Threshold', fontweight='bold')
    ax2.set_ylabel('V-measure', fontweight='bold')
    ax2.set_title('V-measure vs Threshold')
    ax2.grid(True, alpha=0.3)

    ax3 = axes[1, 0]
    prr_vals = results_df['perfect_reconstruction_rate'].values * 100
    norm_vals = normalize(prr_vals)
    colors = plt.cm.viridis(norm_vals)
    ax3.plot(thresholds_pct, prr_vals, marker='o', linewidth=2, markersize=8)
    for i, (x, y) in enumerate(zip(thresholds_pct, prr_vals)):
        ax3.plot(x, y, marker='o', color=colors[i], markersize=10)
        ax3.text(i, y, f'{y:.1f}%', ha='center', va='bottom', fontsize=9)
    ax3.set_xlabel('Similarity Threshold', fontweight='bold')
    ax3.set_ylabel('Perfect Reconstruction Rate (%)', fontweight='bold')
    ax3.set_title('Perfect Reconstruction Rate vs Threshold')
    ax3.grid(True, alpha=0.3)

    ax4 = axes[1, 1]
    n_clusters_vals = results_df['n_clusters'].values
    norm_vals = normalize(n_clusters_vals)
    colors = plt.cm.viridis(norm_vals)
    ax4.plot(thresholds_pct, n_clusters_vals, marker='o', linewidth=2, markersize=8)
    for i, (x, y) in enumerate(zip(thresholds_pct, n_clusters_vals)):
        ax4.plot(x, y, marker='o', color=colors[i], markersize=10)
        ax4.text(i, y, f'{y}', ha='center', va='bottom', fontsize=9)
    ax4.set_xlabel('Similarity Threshold', fontweight='bold')
    ax4.set_ylabel('Number of Poem Clusters', fontweight='bold')
    ax4.set_title('Number of Clusters vs Threshold')
    ax4.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"\nVisualization saved to: {save_path}")
    plt.close()


def verse_level_grid_search(texts, df, thresholds, shingle_sizes, num_perm=128):
    results = []
    best_ari = -1
    best_threshold = None
    best_shingle_size = None
    best_clusters = None
    best_similarities = None

    print("\n" + "="*100)
    print("VERSE-LEVEL 2D GRID SEARCH")
    print("="*100)
    print(f"\nTesting thresholds: {[f'{t:.0%}' for t in thresholds]}")
    print(f"Testing shingle sizes: {shingle_sizes}\n")

    total_combinations = len(thresholds) * len(shingle_sizes)
    print(f"Total combinations: {total_combinations}\n")

    for shingle_size in shingle_sizes:
        for threshold in thresholds:
            print(f"\nTesting shingle_size={shingle_size}, threshold={threshold:.0%}...")

            clusterer = FastMinHashClustering(
                threshold=threshold,
                shingle_size=shingle_size,
                num_perm=num_perm,
                chunk_size=1
            )

            clusters, similarities = clusterer.cluster(texts)

            if 'idgroup' in df.columns:
                temp_df = df.copy()
                temp_df['cluster_id'] = temp_df.index.map(clusters)

                mask = temp_df['idgroup'].notna() & temp_df['cluster_id'].notna()
                y_true = temp_df.loc[mask, 'idgroup'].tolist()
                y_pred = temp_df.loc[mask, 'cluster_id'].tolist()

                ari, v_measure = evaluate_clustering(y_true, y_pred)
                n_gt_clusters = len(set(y_true))
            else:
                ari, v_measure = 0, 0
                n_gt_clusters = 0

            n_clusters = len(set(clusters.values()))

            results.append({
                'shingle_size': shingle_size,
                'threshold': threshold,
                'n_clusters': n_clusters,
                'n_similarities': len(similarities),
                'ari': ari,
                'v_measure': v_measure,
                'n_gt_clusters': n_gt_clusters
            })

            if ari > best_ari:
                best_ari = ari
                best_threshold = threshold
                best_shingle_size = shingle_size
                best_clusters = clusters
                best_similarities = similarities

            print(f"  ARI: {ari:.4f}, V-measure: {v_measure:.4f}, Clusters: {n_clusters}")

    results_df = pd.DataFrame(results)

    print("\n" + "="*100)
    print("VERSE-LEVEL GRID SEARCH SUMMARY")
    print("="*100)
    print(f"\n{'Shingle':<10} {'Threshold':<12} {'Clusters':<10} {'Similarities':<15} {'ARI':<8} {'V-measure':<12}")
    print("-" * 80)

    for _, result in results_df.iterrows():
        print(f"{result['shingle_size']:<10} "
              f"{result['threshold']:<12.0%} "
              f"{result['n_clusters']:<10} "
              f"{result['n_similarities']:<15} "
              f"{result['ari']:<8.4f} "
              f"{result['v_measure']:<12.4f}")

    print(f"\n{'='*100}")
    print("BEST VERSE-LEVEL PARAMETERS")
    print("="*100)
    print(f"\nBest parameters by ARI:")
    print(f"  Shingle size: {best_shingle_size}")
    print(f"  Threshold: {best_threshold:.0%}")
    best_result = results_df[(results_df['threshold'] == best_threshold) &
                              (results_df['shingle_size'] == best_shingle_size)].iloc[0]
    print(f"  ARI: {best_result['ari']:.4f}")
    print(f"  V-measure: {best_result['v_measure']:.4f}")
    print(f"  Number of clusters: {best_result['n_clusters']}")
    print(f"  Number of similarities found: {best_result['n_similarities']}")

    visualize_verse_grid_search(results_df)

    results_csv = RESULTS_DIR / 'verse_grid_search_results.csv'
    results_df.to_csv(results_csv, index=False)
    print(f"\nVerse grid search results saved to: {results_csv}")

    return best_clusters, best_similarities, best_threshold, best_shingle_size, results_df


def poem_level_grid_search(df, poem_to_clusters, thresholds):
    results = []
    best_ari = -1
    best_threshold = None
    best_poem_clusters = None

    print("\n" + "="*100)
    print("POEM-LEVEL GRID SEARCH")
    print("="*100)
    print(f"\nTesting thresholds: {[f'{t:.0%}' for t in thresholds]}\n")

    for threshold in thresholds:
        print(f"\nTesting threshold {threshold:.0%}...")

        poem_clusters, poem_edges, n_clusters = cluster_poems(poem_to_clusters, threshold)

        if 'type_id' in df.columns:
            poem_to_type = df.groupby('idoriginal_poem')['type_id'].first().to_dict()

            y_true = []
            y_pred = []
            for poem_id, predicted_cluster in poem_clusters.items():
                if poem_id in poem_to_type:
                    y_true.append(poem_to_type[poem_id])
                    y_pred.append(predicted_cluster)

            ari, v_measure = evaluate_clustering(y_true, y_pred)
            reconstruction_rate, n_perfect, n_total_gt = calculate_perfect_reconstruction_rate(df, poem_clusters)
        else:
            ari, v_measure = 0, 0
            reconstruction_rate, n_perfect, n_total_gt = 0, 0, 0

        results.append({
            'threshold': threshold,
            'n_clusters': n_clusters,
            'n_edges': len(poem_edges),
            'ari': ari,
            'v_measure': v_measure,
            'perfect_reconstruction_rate': reconstruction_rate,
            'n_perfect_clusters': n_perfect,
            'n_total_gt_clusters': n_total_gt
        })

        if ari > best_ari:
            best_ari = ari
            best_threshold = threshold
            best_poem_clusters = poem_clusters

        print(f"  ARI: {ari:.4f}, V-measure: {v_measure:.4f}, Clusters: {n_clusters}")

    results_df = pd.DataFrame(results)

    print("\n" + "="*100)
    print("POEM-LEVEL GRID SEARCH SUMMARY")
    print("="*100)
    print(f"\n{'Threshold':<12} {'Clusters':<10} {'Edges':<10} {'ARI':<8} {'V-measure':<12} {'Perfect Recon.':<15}")
    print("-" * 80)

    for _, result in results_df.iterrows():
        print(f"{result['threshold']:<12.0%} "
              f"{result['n_clusters']:<10} "
              f"{result['n_edges']:<10} "
              f"{result['ari']:<8.4f} "
              f"{result['v_measure']:<12.4f} "
              f"{result['perfect_reconstruction_rate']:<15.1%}")

    print(f"\n{'='*100}")
    print("BEST POEM-LEVEL THRESHOLD")
    print("="*100)
    print(f"\nBest threshold by ARI: {best_threshold:.0%}")
    best_result = results_df[results_df['threshold'] == best_threshold].iloc[0]
    print(f"  ARI: {best_result['ari']:.4f}")
    print(f"  V-measure: {best_result['v_measure']:.4f}")
    print(f"  Perfect reconstruction rate: {best_result['perfect_reconstruction_rate']:.1%}")
    print(f"    ({best_result['n_perfect_clusters']:.0f}/{best_result['n_total_gt_clusters']:.0f} GT clusters perfectly reconstructed)")

    visualize_poem_grid_search(results_df)

    results_csv = RESULTS_DIR / 'poem_grid_search_results.csv'
    results_df.to_csv(results_csv, index=False)
    print(f"\nPoem grid search results saved to: {results_csv}")

    return best_poem_clusters, best_threshold, results_df


def main():
    DATA_FILE = 'dbbe_full.csv'

    print("="*100)
    print("LOADING DATA")
    print("="*100)
    print(f"Results will be saved to: {RESULTS_DIR}")

    df = pd.read_csv(DATA_FILE)

    if 'verse' in df.columns:
        df['text'] = df['verse']
    elif 'text' not in df.columns:
        raise ValueError("Dataset must have either 'verse' or 'text' column")

    df['text'] = df['text'].fillna('').astype(str)
    print(f"\nLoaded {df.shape[0]:,} verses")

    texts = df['text'].tolist()

    verse_thresholds = [0.2, 0.3, 0.4, 0.5]
    shingle_sizes = [2, 3, 4, 5]

    best_clusters, best_similarities, best_verse_threshold, best_shingle_size, verse_results = verse_level_grid_search(
        texts, df, verse_thresholds, shingle_sizes, num_perm=128
    )

    df['cluster_id'] = df.index.map(best_clusters)

    sim_dict = defaultdict(list)
    for doc1, doc2, sim in best_similarities:
        sim_dict[doc1].append(sim)
        sim_dict[doc2].append(sim)

    df['certainty'] = df.index.map(
        lambda i: np.mean(sim_dict[i]) if i in sim_dict else 1.0
    )

    preprocessor = TextPreprocessor(lowercase=True, remove_punctuation=True, remove_diacritics=True)
    df['text_preprocessed'] = df['text'].apply(preprocessor.preprocess)

    verse_output = RESULTS_DIR / "dbbe_verse_clustered_results.csv"
    df.to_csv(verse_output, index=False)
    print(f"\n{verse_output} saved with best parameters (shingle_size={best_shingle_size}, threshold={best_verse_threshold:.0%})")

    if 'idoriginal_poem' in df.columns and 'type_id' in df.columns:
        poem_to_clusters, poem_verse_counts = reconstruct_poems(df)

        poem_thresholds = [0.50, 0.60, 0.70, 0.8]

        best_poem_clusters, best_poem_threshold, poem_results = poem_level_grid_search(
            df, poem_to_clusters, poem_thresholds
        )

        df['poem_cluster_id'] = df['idoriginal_poem'].map(best_poem_clusters)

        poem_output = RESULTS_DIR / "dbbe_poem_level_clusters.csv"
        df.to_csv(poem_output, index=False)
        print(f"\n{poem_output} saved with best threshold ({best_poem_threshold:.0%})")
    else:
        print("\n" + "="*100)
        print("SKIPPING POEM-LEVEL CLUSTERING")
        print("="*100)
        print("\nRequired columns 'idoriginal_poem' and/or 'type_id' not found")

    print("\n" + "="*100)
    print("ANALYSIS COMPLETE")
    print("="*100)
    print(f"All results saved to: {RESULTS_DIR}")


if __name__ == "__main__":
    main()

GPU detected - using CuPy acceleration
LOADING DATA

Loaded 54,170 verses

VERSE-LEVEL 2D GRID SEARCH

Testing thresholds: ['20%', '30%', '40%', '50%']
Testing shingle sizes: [2, 3, 4, 5]

Total combinations: 16


Testing shingle_size=2, threshold=20%...
Using GPU for MinHash (128 permutations)
Initialized in GPU (CuPy) mode

Clustering 54,170 documents in 54170 chunks
threshold=0.2, chunk_size=1


Processing: 100%|██████████| 54170/54170 [00:51<00:00, 1056.59it/s]



Found 571,991 similarities in 51.27s
Throughput: 1,057 docs/sec
Building clusters...


Clustering: 100%|██████████| 571991/571991 [00:00<00:00, 2156707.47it/s]



Created 20,443 clusters in 51.55s total
  ARI: 0.0800, V-measure: 0.9455, Clusters: 20443

Testing shingle_size=2, threshold=30%...
Using GPU for MinHash (128 permutations)
Initialized in GPU (CuPy) mode

Clustering 54,170 documents in 54170 chunks
threshold=0.3, chunk_size=1


...


Created 28,494 clusters in 52.86s total
  ARI: 0.8570, V-measure: 0.9827, Clusters: 28494

Testing shingle_size=5, threshold=50%...
Using GPU for MinHash (128 permutations)
Initialized in GPU (CuPy) mode

Clustering 54,170 documents in 54170 chunks
threshold=0.5, chunk_size=1


Processing: 100%|██████████| 54170/54170 [00:53<00:00, 1014.94it/s]



Found 422,262 similarities in 53.37s
Throughput: 1,015 docs/sec
Building clusters...


Clustering: 100%|██████████| 422262/422262 [00:00<00:00, 1761605.43it/s]



Created 28,538 clusters in 53.62s total
  ARI: 0.8543, V-measure: 0.9825, Clusters: 28538

VERSE-LEVEL GRID SEARCH SUMMARY

Shingle    Threshold    Clusters   Similarities    ARI      V-measure   
--------------------------------------------------------------------------------
2.0        20%          20443.0    571991.0        0.0800   0.9455      
2.0        30%          22215.0    549960.0        0.1567   0.9653      
2.0        40%          24327.0    537327.0        0.7814   0.9884      
2.0        50%          25057.0    534306.0        0.9258   0.9924      
3.0        20%          26415.0    485262.0        0.9156   0.9905      
3.0        30%          26425.0    485235.0        0.9157   0.9905      
3.0        40%          26453.0    485132.0        0.9156   0.9905      
3.0        50%          26507.0    484836.0        0.9158   0.9906      
4.0        20%          27384.0    455716.0        0.9044   0.9877      
4.0        30%          27389.0    455703.0        0.9044   0.98

In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

RESULTS_DIR = Path("dbbe_orthographic_results")
ANALYSIS_DIR = RESULTS_DIR / "interesting_clusters_analysis"
ANALYSIS_DIR.mkdir(exist_ok=True)

print("="*100)
print("INTERESTING CLUSTER IDENTIFICATION & ANALYSIS")
print("="*100)

verse_file = RESULTS_DIR / "dbbe_verse_clustered_results.csv"
df = pd.read_csv(verse_file)

print(f"\nLoaded {len(df):,} verses")
print(f"Total clusters: {df['cluster_id'].nunique():,}")

print("\n" + "="*100)
print("CLUSTER STATISTICS")
print("="*100)

cluster_stats = []
for cluster_id in df['cluster_id'].unique():
    cluster_df = df[df['cluster_id'] == cluster_id]
    
    n_verses = len(cluster_df)
    
    if 'source_dataset' in df.columns:
        datasets = cluster_df['source_dataset'].unique()
        n_datasets = len(datasets)
        dataset_list = list(datasets)
    else:
        n_datasets = 1
        dataset_list = ['unknown']
    
    if 'idoriginal_poem' in df.columns:
        n_poems = cluster_df['idoriginal_poem'].nunique()
    else:
        n_poems = 0
    
    if 'type_id' in df.columns:
        n_types = cluster_df['type_id'].nunique()
        type_list = cluster_df['type_id'].unique().tolist()
    else:
        n_types = 0
        type_list = []
    
    avg_certainty = cluster_df['certainty'].mean() if 'certainty' in df.columns else 1.0
    
    avg_verse_length = cluster_df['text'].str.len().mean()
    
    cluster_stats.append({
        'cluster_id': cluster_id,
        'n_verses': n_verses,
        'n_datasets': n_datasets,
        'n_poems': n_poems,
        'n_types': n_types,
        'avg_certainty': avg_certainty,
        'avg_verse_length': avg_verse_length,
        'datasets': ','.join(map(str, dataset_list)),
        'types': ','.join(map(str, type_list))
    })

stats_df = pd.DataFrame(cluster_stats)

stats_csv = ANALYSIS_DIR / "cluster_statistics.csv"
stats_df.to_csv(stats_csv, index=False)
print(f"✓ Cluster statistics saved to: {stats_csv}")

print("\n" + "="*100)
print("INTERESTING CLUSTER CATEGORIES")
print("="*100)

interesting_clusters = defaultdict(list)

cross_dataset_clusters = stats_df[stats_df['n_datasets'] > 1].sort_values('n_verses', ascending=False)
print(f"\n1. CROSS-DATASET CLUSTERS: {len(cross_dataset_clusters)}")
print("   (Verses appearing in multiple datasets - potential formulaic expressions)")
if len(cross_dataset_clusters) > 0:
    top_cross = cross_dataset_clusters.head(10)
    print(f"\n   Top 10 by size:")
    for _, row in top_cross.iterrows():
        print(f"   Cluster {row['cluster_id']}: {row['n_verses']} verses across {row['n_datasets']} datasets")
    interesting_clusters['cross_dataset'] = cross_dataset_clusters['cluster_id'].tolist()

large_clusters = stats_df[stats_df['n_verses'] >= 5].sort_values('n_verses', ascending=False)
print(f"\n2. LARGE CLUSTERS (≥5 verses): {len(large_clusters)}")
print("   (Frequently repeated verses - common formulaic patterns)")
if len(large_clusters) > 0:
    top_large = large_clusters.head(10)
    print(f"\n   Top 10 by size:")
    for _, row in top_large.iterrows():
        print(f"   Cluster {row['cluster_id']}: {row['n_verses']} verses")
    interesting_clusters['large'] = large_clusters['cluster_id'].tolist()

if 'n_types' in stats_df.columns:
    cross_type_clusters = stats_df[(stats_df['n_types'] > 1) & (stats_df['n_verses'] >= 3)].sort_values('n_verses', ascending=False)
    print(f"\n3. CROSS-TYPE CLUSTERS: {len(cross_type_clusters)}")
    print("   (Verses shared across different poem types - borrowing/influence)")
    if len(cross_type_clusters) > 0:
        top_cross_type = cross_type_clusters.head(10)
        print(f"\n   Top 10 by size:")
        for _, row in top_cross_type.iterrows():
            print(f"   Cluster {row['cluster_id']}: {row['n_verses']} verses across {row['n_types']} types")
        interesting_clusters['cross_type'] = cross_type_clusters['cluster_id'].tolist()

high_certainty_clusters = stats_df[(stats_df['avg_certainty'] >= 0.8) & (stats_df['n_verses'] >= 3)].sort_values('avg_certainty', ascending=False)
print(f"\n4. HIGH-CERTAINTY CLUSTERS: {len(high_certainty_clusters)}")
print("   (Very similar verses - exact/near-exact repetitions)")
if len(high_certainty_clusters) > 0:
    top_certainty = high_certainty_clusters.head(10)
    print(f"\n   Top 10 by certainty:")
    for _, row in top_certainty.iterrows():
        print(f"   Cluster {row['cluster_id']}: {row['n_verses']} verses, certainty={row['avg_certainty']:.3f}")
    interesting_clusters['high_certainty'] = high_certainty_clusters['cluster_id'].tolist()

medium_clusters = stats_df[(stats_df['n_verses'] >= 3) & (stats_df['n_verses'] <= 10)]
print(f"\n5. MEDIUM-SIZED CLUSTERS (3-10 verses): {len(medium_clusters)}")
print("   (Moderately repeated - good for detailed case studies)")
if len(medium_clusters) > 0:
    interesting_clusters['medium'] = medium_clusters['cluster_id'].tolist()

print("\n" + "="*100)
print("GENERATING DETAILED REPORTS FOR TOP CLUSTERS")
print("="*100)

def generate_cluster_report(cluster_id, df, output_file):
    cluster_df = df[df['cluster_id'] == cluster_id].copy()
    
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("="*100 + "\n")
        f.write(f"CLUSTER {cluster_id} - DETAILED REPORT\n")
        f.write("="*100 + "\n\n")
        
        f.write(f"Number of verses: {len(cluster_df)}\n")
        
        if 'source_dataset' in cluster_df.columns:
            f.write(f"Datasets: {cluster_df['source_dataset'].unique().tolist()}\n")
            f.write(f"Dataset distribution:\n")
            for dataset, count in cluster_df['source_dataset'].value_counts().items():
                f.write(f"  - {dataset}: {count} verses\n")
        
        if 'idoriginal_poem' in cluster_df.columns:
            f.write(f"\nNumber of poems: {cluster_df['idoriginal_poem'].nunique()}\n")
        
        if 'type_id' in cluster_df.columns:
            f.write(f"Poem types: {cluster_df['type_id'].unique().tolist()}\n")
        
        if 'certainty' in cluster_df.columns:
            f.write(f"\nAverage certainty: {cluster_df['certainty'].mean():.3f}\n")
            f.write(f"Certainty range: {cluster_df['certainty'].min():.3f} - {cluster_df['certainty'].max():.3f}\n")
        
        f.write(f"\nAverage verse length: {cluster_df['text'].str.len().mean():.1f} characters\n")
        
        f.write("\n" + "="*100 + "\n")
        f.write("ALL VERSES IN CLUSTER\n")
        f.write("="*100 + "\n\n")
        
        for idx, row in cluster_df.iterrows():
            f.write(f"Verse {idx}:\n")
            f.write(f"  Text: {row['text']}\n")
            if 'text_preprocessed' in row:
                f.write(f"  Preprocessed: {row['text_preprocessed']}\n")
            if 'source_dataset' in row:
                f.write(f"  Dataset: {row['source_dataset']}\n")
            if 'idoriginal_poem' in row:
                f.write(f"  Poem ID: {row['idoriginal_poem']}\n")
            if 'type_id' in row:
                f.write(f"  Type: {row['type_id']}\n")
            if 'certainty' in row:
                f.write(f"  Certainty: {row['certainty']:.3f}\n")
            f.write("\n")
        
        if 'text_preprocessed' in cluster_df.columns:
            f.write("="*100 + "\n")
            f.write("UNIQUE PREPROCESSED FORMS\n")
            f.write("="*100 + "\n\n")
            unique_forms = cluster_df['text_preprocessed'].value_counts()
            for form, count in unique_forms.items():
                f.write(f"{form} (appears {count} times)\n\n")

n_reports = 5
print(f"\nGenerating detailed reports for top {n_reports} clusters in each category...\n")

for category, cluster_ids in interesting_clusters.items():
    category_dir = ANALYSIS_DIR / category
    category_dir.mkdir(exist_ok=True)
    
    top_clusters = cluster_ids[:n_reports]
    
    print(f"Category: {category.upper()}")
    for i, cluster_id in enumerate(top_clusters, 1):
        output_file = category_dir / f"cluster_{cluster_id}_report.txt"
        generate_cluster_report(cluster_id, df, output_file)
        print(f"  {i}. Cluster {cluster_id} → {output_file}")

print("\n" + "="*100)
print("CREATING VISUALIZATIONS")
print("="*100)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Cluster Analysis Overview', fontsize=16, fontweight='bold')

ax1 = axes[0, 0]
cluster_size_dist = stats_df['n_verses'].value_counts().sort_index()
ax1.bar(cluster_size_dist.index, cluster_size_dist.values, color='#0173B2', alpha=0.7)
ax1.set_xlabel('Cluster Size (# verses)', fontweight='bold')
ax1.set_ylabel('Frequency', fontweight='bold')
ax1.set_title('Cluster Size Distribution')
ax1.set_xlim(0, min(50, cluster_size_dist.index.max()))
ax1.grid(alpha=0.3)

ax2 = axes[0, 1]
if 'n_datasets' in stats_df.columns:
    dataset_dist = stats_df['n_datasets'].value_counts().sort_index()
    ax2.bar(dataset_dist.index, dataset_dist.values, color='#ECB01F', alpha=0.7)
    ax2.set_xlabel('Number of Datasets', fontweight='bold')
    ax2.set_ylabel('Number of Clusters', fontweight='bold')
    ax2.set_title('Cross-Dataset Distribution')
    ax2.grid(alpha=0.3)

ax3 = axes[1, 0]
if 'avg_certainty' in stats_df.columns:
    ax3.hist(stats_df['avg_certainty'], bins=30, color='#029E73', alpha=0.7, edgecolor='black')
    ax3.axvline(stats_df['avg_certainty'].mean(), color='red', linestyle='--', 
                linewidth=2, label=f'Mean: {stats_df["avg_certainty"].mean():.3f}')
    ax3.set_xlabel('Average Certainty', fontweight='bold')
    ax3.set_ylabel('Number of Clusters', fontweight='bold')
    ax3.set_title('Certainty Distribution')
    ax3.legend()
    ax3.grid(alpha=0.3)

ax4 = axes[1, 1]
top_20 = stats_df.nlargest(20, 'n_verses')
ax4.barh(range(len(top_20)), top_20['n_verses'].values, color='#CC0000', alpha=0.7)
ax4.set_yticks(range(len(top_20)))
ax4.set_yticklabels([f"Cluster {cid}" for cid in top_20['cluster_id'].values], fontsize=8)
ax4.set_xlabel('Number of Verses', fontweight='bold')
ax4.set_title('Top 20 Largest Clusters')
ax4.invert_yaxis()
ax4.grid(alpha=0.3, axis='x')

plt.tight_layout()
overview_plot = ANALYSIS_DIR / "cluster_analysis_overview.png"
plt.savefig(overview_plot, dpi=300, bbox_inches='tight')
plt.close()
print(f"\n✓ Overview visualization saved: {overview_plot}")

print("\n" + "="*100)
print("CREATING SUMMARY TABLES")
print("="*100)

summary_file = ANALYSIS_DIR / "interesting_clusters_summary.txt"
with open(summary_file, 'w', encoding='utf-8') as f:
    f.write("="*100 + "\n")
    f.write("INTERESTING CLUSTERS SUMMARY\n")
    f.write("="*100 + "\n\n")
    
    for category, cluster_ids in interesting_clusters.items():
        f.write(f"\n{category.upper().replace('_', ' ')} ({len(cluster_ids)} clusters)\n")
        f.write("-"*100 + "\n")
        
        category_stats = stats_df[stats_df['cluster_id'].isin(cluster_ids[:20])]
        f.write(f"{'Cluster ID':<12} {'Verses':<8} {'Datasets':<10} {'Poems':<8} {'Types':<8} {'Certainty':<10}\n")
        f.write("-"*100 + "\n")
        
        for _, row in category_stats.iterrows():
            f.write(f"{row['cluster_id']:<12} "
                   f"{row['n_verses']:<8} "
                   f"{row['n_datasets']:<10} "
                   f"{row['n_poems']:<8} "
                   f"{row['n_types']:<8} "
                   f"{row['avg_certainty']:<10.3f}\n")
        f.write("\n")


INTERESTING CLUSTER IDENTIFICATION & ANALYSIS

Loaded 54,170 verses
Total clusters: 25,057

CLUSTER STATISTICS
✓ Cluster statistics saved to: dbbe_orthographic_results/interesting_clusters_analysis/cluster_statistics.csv

INTERESTING CLUSTER CATEGORIES

1. CROSS-DATASET CLUSTERS: 0
   (Verses appearing in multiple datasets - potential formulaic expressions)

2. LARGE CLUSTERS (≥5 verses): 1756
   (Frequently repeated verses - common formulaic patterns)

   Top 10 by size:
   Cluster 459: 350 verses
   Cluster 7389: 297 verses
   Cluster 1065: 245 verses
   Cluster 1901: 226 verses
   Cluster 62: 207 verses
   Cluster 454: 197 verses
   Cluster 2315: 177 verses
   Cluster 3246: 176 verses
   Cluster 2439: 175 verses
   Cluster 2880: 174 verses

3. CROSS-TYPE CLUSTERS: 822
   (Verses shared across different poem types - borrowing/influence)

   Top 10 by size:
   Cluster 459: 350 verses across 250 types
   Cluster 7389: 297 verses across 30 types
   Cluster 1065: 245 verses across 37 typ

# 2. Full dataset

In [5]:
import re
import unicodedata
from typing import Dict
import pandas as pd
import numpy as np
from tqdm import tqdm
from datasketch import MinHash, MinHashLSHForest
import multiprocessing as mp
import hashlib
import time
import matplotlib.pyplot as plt
import seaborn as sns
CLEAN_PATTERN = re.compile(r'[^\w\s]')
WHITESPACE_PATTERN = re.compile(r'\s+')

def preprocess_text(text: str, options: Dict[str, bool] = None) -> str:
    if options is None:
        options = {'lowercase': True, 'remove_diacritics': True, 'remove_punctuation': True}
    text = str(text)
    if options.get('lowercase', True):
        text = text.lower()
    if options.get('remove_diacritics', True):
        text = unicodedata.normalize('NFD', text)
        text = ''.join(char for char in text if unicodedata.category(char) != 'Mn')
        text = unicodedata.normalize('NFC', text)
    else:
        text = unicodedata.normalize('NFC', text)
    if options.get('remove_punctuation', True):
        text = CLEAN_PATTERN.sub('', text)
    text = WHITESPACE_PATTERN.sub(' ', text)
    return text.strip()

class FastUnionFind:
    __slots__ = ['parent', 'rank']
    
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n
    
    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def union(self, x, y):
        px, py = self.find(x), self.find(y)
        if px == py:
            return False
        if self.rank[px] < self.rank[py]:
            self.parent[px] = py
        elif self.rank[px] > self.rank[py]:
            self.parent[py] = px
        else:
            self.parent[py] = px
            self.rank[px] += 1
        return True
    
    def get_clusters(self):
        return np.array([self.find(i) for i in range(len(self.parent))], dtype=np.int32)

def get_ngrams_vectorized(text, n=4):
    if not text or len(text) < n:
        return set()
    text = str(text).lower()
    return set(text[i:i+n] for i in range(len(text)-n+1))

def compute_minhash_chunk(args):
    texts, start_idx, n_gram_size, num_perm, seed = args
    np.random.seed(seed)
    minhashes = []
    for text in texts:
        ngrams = get_ngrams_vectorized(text, n_gram_size)
        m = MinHash(num_perm=num_perm, seed=seed)
        if ngrams:
            for ngram in ngrams:
                m.update(ngram.encode('utf8'))
        minhashes.append(m)
    return minhashes

def compute_minhash_parallel(texts, n_gram_size=3, num_perm=128, n_cores=None):
    if n_cores is None:
        n_cores = min(mp.cpu_count(), 16)
    chunk_size = max(100, min(2000, len(texts) // (n_cores * 2)))
    chunks = [(texts[i:i+chunk_size], i, n_gram_size, num_perm, 42) 
              for i in range(0, len(texts), chunk_size)]
    with mp.Pool(n_cores) as pool:
        results = list(tqdm(pool.imap(compute_minhash_chunk, chunks),
                          total=len(chunks), desc="MinHash"))
    minhashes = [mh for chunk_mhs in results for mh in chunk_mhs]
    return minhashes

def fast_hash(data):
    return int(hashlib.md5(data).hexdigest()[:16], 16)

def find_exact_duplicates_fast(texts):
    text_hashes = {}
    for i, text in enumerate(tqdm(texts, desc="Hashing", disable=len(texts)<100000)):
        normalized = str(text).strip().lower()
        if not normalized:
            continue
        text_hash = fast_hash(normalized.encode('utf-8'))
        text_hashes.setdefault(text_hash, []).append(i)
    duplicate_groups = [indices for indices in text_hashes.values() if len(indices) > 1]
    return duplicate_groups

def stratified_sample(df, n_sample=15000):
    datasets = df['source_dataset'].unique()
    total_size = len(df)
    sample_indices = []
    
    for dataset in datasets:
        dataset_indices = df[df['source_dataset'] == dataset].index.tolist()
        dataset_size = len(dataset_indices)
        proportion = dataset_size / total_size
        n_from_dataset = int(n_sample * proportion)
        n_from_dataset = min(n_from_dataset, dataset_size)
        sampled = np.random.choice(dataset_indices, size=n_from_dataset, replace=False)
        sample_indices.extend(sampled)
    
    return sorted(sample_indices)

def quick_threshold_selection(texts, minhashes, df, n_sample=15000):
    sample_indices = stratified_sample(df, n_sample)
    sample_size = len(sample_indices)
    
    forest = MinHashLSHForest(num_perm=len(minhashes[0].hashvalues))
    for idx in tqdm(sample_indices, desc="Indexing sample", leave=False):
        forest.add(str(idx), minhashes[idx])
    forest.index()
    
    k = 20
    similarities = []
    for idx in tqdm(sample_indices, desc="Querying sample", leave=False):
        neighbors = forest.query(minhashes[idx], k)
        for neighbor_idx_str in neighbors[1:]:
            neighbor_idx = int(neighbor_idx_str)
            sim = minhashes[idx].jaccard(minhashes[neighbor_idx])
            similarities.append(sim)
    
    similarities = np.array(similarities)
    
    if np.percentile(similarities, 95) > 0.5:
        threshold = np.percentile(similarities, 85)
    elif np.percentile(similarities, 95) > 0.3:
        threshold = np.percentile(similarities, 80)
    else:
        threshold = np.percentile(similarities, 75)
    
    threshold = max(0.3, min(0.85, threshold))

    
    sns.set_palette("colorblind")
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    ax1.hist(similarities, bins=50, color='#0173B2', alpha=0.7, edgecolor='black')
    ax1.axvline(threshold, color='#CC0000', linestyle='--', linewidth=2, label=f'Threshold: {threshold:.3f}')
    ax1.set_xlabel('Jaccard Similarity', fontweight='bold')
    ax1.set_ylabel('Frequency', fontweight='bold')
    ax1.set_title('Similarity Distribution', fontweight='bold')
    ax1.legend()
    ax1.grid(alpha=0.3)
    
    n_above = np.sum(similarities >= threshold)
    pct_above = (n_above / len(similarities)) * 100
    
    ax2.hist(similarities, bins=50, color='#0173B2', alpha=0.7, edgecolor='black', cumulative=True, density=True)
    ax2.axvline(threshold, color='#CC0000', linestyle='--', linewidth=2, label=f'Threshold: {threshold:.3f}')
    ax2.axhline(0.85, color='gray', linestyle=':', alpha=0.5, label='85th percentile')
    ax2.set_xlabel('Jaccard Similarity', fontweight='bold')
    ax2.set_ylabel('Cumulative Probability', fontweight='bold')
    ax2.set_title('Cumulative Distribution', fontweight='bold')
    ax2.legend()
    ax2.grid(alpha=0.3)
    
    fig.suptitle(f'Threshold Selection (n={len(similarities):,} pairs, {pct_above:.1f}% kept)', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('threshold_selection.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return threshold

def cluster_with_lsh_forest(minhashes, duplicate_groups, threshold, top_k=100):
    n_docs = len(minhashes)
    uf = FastUnionFind(n_docs)
    
    exact_merges = 0
    for group in duplicate_groups:
        for i in range(1, len(group)):
            if uf.union(group[0], group[i]):
                exact_merges += 1
    
    forest = MinHashLSHForest(num_perm=len(minhashes[0].hashvalues))
    for idx, mh in enumerate(tqdm(minhashes, desc="Indexing", leave=False)):
        forest.add(str(idx), mh)
    forest.index()
    
    lsh_merges = 0
    verified_pairs = 0
    chunk_size = 10000
    
    for start_idx in tqdm(range(0, n_docs, chunk_size), desc="Clustering"):
        end_idx = min(start_idx + chunk_size, n_docs)
        for idx in range(start_idx, end_idx):
            if uf.find(idx) != idx:
                continue
            neighbors = forest.query(minhashes[idx], top_k)
            for neighbor_str in neighbors[1:]:
                neighbor_idx = int(neighbor_str)
                if uf.find(idx) == uf.find(neighbor_idx):
                    continue
                verified_pairs += 1
                sim = minhashes[idx].jaccard(minhashes[neighbor_idx])
                if sim >= threshold:
                    if uf.union(idx, neighbor_idx):
                        lsh_merges += 1
    
    cluster_labels = uf.get_clusters()
    unique_clusters, cluster_sizes = np.unique(cluster_labels, return_counts=True)
    
    return cluster_labels, {
        'n_clusters': len(unique_clusters),
        'n_multi_clusters': np.sum(cluster_sizes > 1),
        'n_singletons': np.sum(cluster_sizes == 1),
        'avg_cluster_size': float(cluster_sizes.mean()),
        'max_cluster_size': int(cluster_sizes.max()),
        'exact_merges': exact_merges,
        'lsh_merges': lsh_merges,
        'threshold': threshold,
        'verified_pairs': verified_pairs
    }

def main():
    df = pd.read_csv("concatenated.csv")
    df = df[df['source_dataset'].isin(['rhoby', 'dbbe', 'phi', 'papyri'])]
    df = df[df['verse'].fillna('').astype(str).str.len() >= 20]
    df['verse'] = df['verse'].apply(preprocess_text)
    df = df.reset_index(drop=True)
    df = df[df['verse'].str.strip().str.lower() != 'nan']
    texts = df['verse'].fillna('').astype(str).tolist()
    
    print(f"Verses: {len(texts):,}")
    
    start_time = time.time()
    
    duplicate_groups = find_exact_duplicates_fast(texts)
    t1 = time.time()
    
    minhashes = compute_minhash_parallel(texts, n_gram_size=3, num_perm=128)
    t2 = time.time()
    
    threshold = quick_threshold_selection(texts, minhashes, df, n_sample=15000)
    t3 = time.time()
    print(f"Threshold: {threshold:.3f}")
    
    cluster_labels, metrics = cluster_with_lsh_forest(
        minhashes, duplicate_groups, threshold, top_k=100
    )
    t4 = time.time()
    
    total_time = time.time() - start_time
    
    df['cluster_id'] = cluster_labels
    df.to_csv("clustered_ultrafast.csv", index=False)
    
    metrics['total_time_minutes'] = total_time / 60
    pd.DataFrame([metrics]).to_csv("clustering_metrics.csv", index=False)
    
    print(f"Time: {total_time/60:.1f} min")
    print(f"Clusters: {metrics['n_multi_clusters']:,}")
    print(f"Max size: {metrics['max_cluster_size']}")

if __name__ == "__main__":
    main()

/tmp/ipykernel_682239/2073978938.py:234: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("concatenated.csv")


Verses: 1,537,740


MinHash: 100%|██████████| 769/769 [02:30<00:00,  5.10it/s]


Threshold: 0.300


Clustering: 100%|██████████| 154/154 [15:18<00:00,  5.96s/it]         


Time: 19.1 min
Clusters: 58,593
Max size: 1083331


In [8]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from numba import njit
import gc

class FastPoemClusterer:
    def __init__(self, threshold: float = 0.6):
        self.threshold = threshold
        self.poem_clusters = {}
        self.similarities = []
    
    @staticmethod
    def reconstruct_poems_vectorized(df):
        """Vectorized poem reconstruction - 10x faster"""
        # Filter out unclustered verses once
        valid_mask = df['cluster_id'] != -1
        df_valid = df[valid_mask]
        
        # Group by poem efficiently
        grouped = df_valid.groupby('idoriginal_poem')['cluster_id'].apply(set)
        return grouped.to_dict()
    
    @staticmethod
    def build_inverted_index_fast(poem_to_clusters):
        """Fast inverted index with list comprehension"""
        cluster_to_poems = defaultdict(set)
        
        # Flatten and build in one pass
        for poem_id, clusters in poem_to_clusters.items():
            for c in clusters:
                cluster_to_poems[c].add(poem_id)
        
        return cluster_to_poems
    
    @staticmethod
    @njit
    def jaccard_numba(a_arr, b_arr):
        """Ultra-fast Jaccard using Numba"""
        intersection = 0
        a_set = set(a_arr)
        b_set = set(b_arr)
        
        for item in a_set:
            if item in b_set:
                intersection += 1
        
        union = len(a_set) + len(b_set) - intersection
        if union == 0:
            return 0.0
        return intersection / union
    
    def find_candidate_pairs_optimized(self, poem_to_clusters, cluster_to_poems):
        """
        Optimized candidate generation:
        - Skip clusters with only 1 poem
        - Pre-allocate set
        - Use batch processing
        """
        candidate_pairs = set()
        
        # Filter out singleton clusters immediately
        multi_poem_clusters = {
            c: poems for c, poems in cluster_to_poems.items() 
            if len(poems) > 1
        }
        
        print(f"Processing {len(multi_poem_clusters):,} clusters (filtered from {len(cluster_to_poems):,})")
        
        # Batch process large clusters differently
        small_threshold = 1000
        
        for cluster_id, poems in tqdm(multi_poem_clusters.items(), desc="Building candidates"):
            poems_list = list(poems)
            n_poems = len(poems_list)
            
            if n_poems <= small_threshold:
                # Small cluster: all pairs
                for i in range(n_poems):
                    for j in range(i + 1, n_poems):
                        candidate_pairs.add(tuple(sorted((poems_list[i], poems_list[j]))))
            else:
                # Large cluster: sample or warn
                print(f"  Warning: Large cluster {cluster_id} with {n_poems} poems - sampling pairs")
                # Sample pairs from large clusters to avoid memory explosion
                sample_size = min(10000, n_poems * (n_poems - 1) // 2)
                indices = np.random.choice(n_poems, size=min(2000, n_poems), replace=False)
                for i in range(len(indices)):
                    for j in range(i + 1, len(indices)):
                        candidate_pairs.add(tuple(sorted((poems_list[indices[i]], poems_list[indices[j]]))))
        
        return candidate_pairs
    
    def cluster_poems_fast(self, df):
        """
        Fast poem clustering with multiple optimizations:
        1. Vectorized data loading
        2. Filtered candidate generation
        3. Numpy-based similarity computation
        4. Optimized Union-Find
        """
        print("Reconstructing poems...")
        poem_to_clusters = self.reconstruct_poems_vectorized(df)
        print(f"Found {len(poem_to_clusters):,} poems")
        
        print("Building inverted index...")
        cluster_to_poems = self.build_inverted_index_fast(poem_to_clusters)
        print(f"Found {len(cluster_to_poems):,} clusters")
        
        print("Finding candidate pairs...")
        candidate_pairs = self.find_candidate_pairs_optimized(poem_to_clusters, cluster_to_poems)
        print(f"Found {len(candidate_pairs):,} candidate pairs")
        
        # Convert sets to sorted arrays for faster Jaccard
        print("Converting to arrays...")
        poem_to_array = {
            p: np.array(sorted(clusters), dtype=np.int32) 
            for p, clusters in poem_to_clusters.items()
        }
        
        # Optimized Union-Find with path compression and union by rank
        poem_ids = list(poem_to_clusters.keys())
        parent = {p: p for p in poem_ids}
        rank = {p: 0 for p in poem_ids}
        
        def find(x):
            if parent[x] != x:
                parent[x] = find(parent[x])  # Path compression
            return parent[x]
        
        def union(x, y):
            px, py = find(x), find(y)
            if px == py:
                return False
            
            # Union by rank
            if rank[px] < rank[py]:
                px, py = py, px
            parent[py] = px
            if rank[px] == rank[py]:
                rank[px] += 1
            return True
        
        # Batch process similarities
        print("Computing similarities and clustering...")
        similarities = []
        merges = 0
        
        # Process in batches for better memory management
        batch_size = 100000
        pairs_list = list(candidate_pairs)
        
        for batch_start in tqdm(range(0, len(pairs_list), batch_size), desc="Processing batches"):
            batch_end = min(batch_start + batch_size, len(pairs_list))
            batch_pairs = pairs_list[batch_start:batch_end]
            
            for p1, p2 in batch_pairs:
                # Skip if already in same cluster
                if find(p1) == find(p2):
                    continue
                
                # Compute Jaccard with Numba-accelerated function
                sim = self.jaccard_numba(poem_to_array[p1], poem_to_array[p2])
                
                if sim >= self.threshold:
                    if union(p1, p2):
                        merges += 1
                    similarities.append((p1, p2, sim))
            
            # Periodic garbage collection
            if batch_start % (batch_size * 5) == 0:
                gc.collect()
        
        print(f"Merged {merges:,} poem pairs")
        
        # Final cluster assignment
        self.poem_clusters = {p: find(p) for p in poem_ids}
        self.similarities = similarities
        
        return self.poem_clusters, self.similarities
    
    def cluster_sizes(self):
        """Return distribution of poem cluster sizes"""
        cluster_count = defaultdict(int)
        for cluster in self.poem_clusters.values():
            cluster_count[cluster] += 1
        sizes = np.array(list(cluster_count.values()))
        return sizes


# =========================
# EVEN FASTER: Pre-filter with size heuristic
# =========================
class UltraFastPoemClusterer(FastPoemClusterer):
    """
    Additional optimization: Pre-filter candidate pairs by size difference.
    If two poems have very different numbers of verse clusters,
    they can't have high Jaccard similarity.
    """
    
    def find_candidate_pairs_with_size_filter(self, poem_to_clusters, cluster_to_poems):
        """
        Size-aware candidate filtering:
        - Only consider pairs where size ratio is within reasonable bounds
        - For threshold=0.6, max size ratio is ~2.5x
        """
        # Calculate size bounds based on threshold
        # For Jaccard >= threshold, max size ratio is (1+threshold)/threshold
        max_ratio = (1 + self.threshold) / self.threshold
        
        candidate_pairs = set()
        
        # Get poem sizes
        poem_sizes = {p: len(clusters) for p, clusters in poem_to_clusters.items()}
        
        # Filter singleton clusters
        multi_poem_clusters = {
            c: poems for c, poems in cluster_to_poems.items() 
            if len(poems) > 1
        }
        
        print(f"Processing {len(multi_poem_clusters):,} clusters")
        print(f"Max size ratio filter: {max_ratio:.2f}x")
        
        filtered_by_size = 0
        
        for cluster_id, poems in tqdm(multi_poem_clusters.items(), desc="Building candidates"):
            poems_list = list(poems)
            n_poems = len(poems_list)
            
            if n_poems > 1000:
                print(f"  Warning: Large cluster {cluster_id} with {n_poems} poems")
                # Sample from large clusters
                poems_list = list(np.random.choice(poems_list, size=min(1000, n_poems), replace=False))
                n_poems = len(poems_list)
            
            for i in range(n_poems):
                p1 = poems_list[i]
                size1 = poem_sizes[p1]
                
                for j in range(i + 1, n_poems):
                    p2 = poems_list[j]
                    size2 = poem_sizes[p2]
                    
                    # Size filter: skip if sizes too different
                    ratio = max(size1, size2) / max(min(size1, size2), 1)
                    if ratio > max_ratio:
                        filtered_by_size += 1
                        continue
                    
                    candidate_pairs.add(tuple(sorted((p1, p2))))
        
        print(f"Filtered {filtered_by_size:,} pairs by size")
        
        return candidate_pairs
    
    def cluster_poems_ultrafast(self, df):
        """Ultra-fast clustering with size-based pre-filtering"""
        print("Reconstructing poems...")
        poem_to_clusters = self.reconstruct_poems_vectorized(df)
        print(f"Found {len(poem_to_clusters):,} poems")
        
        print("Building inverted index...")
        cluster_to_poems = self.build_inverted_index_fast(poem_to_clusters)
        print(f"Found {len(cluster_to_poems):,} clusters")
        
        print("Finding candidate pairs with size filter...")
        candidate_pairs = self.find_candidate_pairs_with_size_filter(
            poem_to_clusters, cluster_to_poems
        )
        print(f"Found {len(candidate_pairs):,} candidate pairs")
        
        # Use the fast clustering from parent class
        return self._cluster_with_pairs(poem_to_clusters, candidate_pairs)
    
    def _cluster_with_pairs(self, poem_to_clusters, candidate_pairs):
        """Internal method to cluster given candidate pairs"""
        # Convert to arrays
        poem_to_array = {
            p: np.array(sorted(clusters), dtype=np.int32) 
            for p, clusters in poem_to_clusters.items()
        }
        
        # Union-Find
        poem_ids = list(poem_to_clusters.keys())
        parent = {p: p for p in poem_ids}
        rank = {p: 0 for p in poem_ids}
        
        def find(x):
            if parent[x] != x:
                parent[x] = find(parent[x])
            return parent[x]
        
        def union(x, y):
            px, py = find(x), find(y)
            if px == py:
                return False
            if rank[px] < rank[py]:
                px, py = py, px
            parent[py] = px
            if rank[px] == rank[py]:
                rank[px] += 1
            return True
        
        # Compute similarities
        print("Computing similarities...")
        similarities = []
        merges = 0
        
        pairs_list = list(candidate_pairs)
        
        for p1, p2 in tqdm(pairs_list, desc="Clustering"):
            if find(p1) == find(p2):
                continue
            
            sim = self.jaccard_numba(poem_to_array[p1], poem_to_array[p2])
            
            if sim >= self.threshold:
                if union(p1, p2):
                    merges += 1
                similarities.append((p1, p2, sim))
        
        print(f"Merged {merges:,} poem pairs")
        
        self.poem_clusters = {p: find(p) for p in poem_ids}
        self.similarities = similarities
        
        return self.poem_clusters, self.similarities


# =========================
# USAGE
# =========================
if __name__ == "__main__":
    import time
    
    print("Loading data...")
    df = pd.read_csv("full_verse_level_ortho.csv")
    df['cluster_id'] = pd.to_numeric(df['cluster_id'], errors='coerce').fillna(-1).astype(int)
    df['idoriginal_poem'] = df['idoriginal_poem'].astype(str)
    
    print(f"Loaded {len(df):,} verses")
    
    # Choose clusterer based on data size
    n_unique_poems = df['idoriginal_poem'].nunique()
    
    if n_unique_poems > 100000:
        print("\nUsing UltraFastPoemClusterer (size-filtered)")
        clusterer = UltraFastPoemClusterer(threshold=0.6)
        start = time.time()
        poem_clusters, similarities = clusterer.cluster_poems_ultrafast(df)
    else:
        print("\nUsing FastPoemClusterer")
        clusterer = FastPoemClusterer(threshold=0.6)
        start = time.time()
        poem_clusters, similarities = clusterer.cluster_poems_fast(df)
    
    elapsed = time.time() - start
    
    sizes = clusterer.cluster_sizes()
    
    print(f"\n{'='*70}")
    print("RESULTS")
    print(f"{'='*70}")
    print(f"Time:                  {elapsed/60:.1f} minutes")
    print(f"Poem clusters:         {len(set(poem_clusters.values())):,}")
    print(f"Candidate pairs:       {len(similarities):,}")
    print(f"Cluster size mean:     {sizes.mean():.2f}")
    print(f"Cluster size median:   {np.median(sizes):.0f}")
    print(f"Cluster size max:      {sizes.max()}")
    print(f"{'='*70}")

Loading data...


/tmp/ipykernel_3364275/1821844082.py:339: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("full_verse_level_ortho.csv")


Loaded 1,537,740 verses

Using UltraFastPoemClusterer (size-filtered)
Reconstructing poems...
Found 164,665 poems
Building inverted index...
Found 310,769 clusters
Finding candidate pairs with size filter...
Processing 34,480 clusters
Max size ratio filter: 2.67x


Building candidates:   0%|          | 0/34480 [00:00<?, ?it/s]

Building candidates:   0%|          | 7/34480 [00:00<28:25, 20.22it/s]

Building candidates: 100%|██████████| 34480/34480 [00:00<00:00, 35282.09it/s]


Filtered 474,862 pairs by size
Found 839,749 candidate pairs
Computing similarities...


Clustering: 100%|██████████| 839749/839749 [00:03<00:00, 252998.86it/s]


Merged 10,810 poem pairs

RESULTS
Time:                  0.1 minutes
Poem clusters:         153,855
Candidate pairs:       10,810
Cluster size mean:     1.07
Cluster size median:   1
Cluster size max:      957


In [ ]:
from collections import defaultdict

# Invert the mapping: cluster_id -> list of poem_ids
clusters_dict = defaultdict(list)
for poem_id, cluster_id in poem_clusters.items():
    clusters_dict[cluster_id].append(poem_id)

for cluster_id, poems in list(clusters_dict.items())[:3]:
    print(f"\nCluster {cluster_id}:")
    verses = df[df['idoriginal_poem'].isin(poems)]['verse'].tolist()
    for v in verses[:5]:  # show first 5 verses in cluster
        print(f"  - {v}")



In [14]:
# Average Jaccard similarity per cluster
cluster_sim_avgs = defaultdict(list)
for p1, p2, sim in similarities:
    cluster_id = poem_clusters[p1]
    cluster_sim_avgs[cluster_id].append(sim)

avg_sim_per_cluster = {cid: np.mean(sims) for cid, sims in cluster_sim_avgs.items()}
print(f"Example: Average similarity in first 5 clusters")
for cid, avg in list(avg_sim_per_cluster.items())[:5]:
    print(f"  Cluster {cid}: avg similarity {avg:.3f}")


Example: Average similarity in first 5 clusters
  Cluster 28467: avg similarity 0.750
  Cluster 396556: avg similarity 1.000
  Cluster 178269: avg similarity 0.803
  Cluster 835623: avg similarity 0.789
  Cluster 19956: avg similarity 0.766


In [ ]:
# Filter clusters with more than one poem AND at least two different sources
multi_source_clusters = {}
for cid, poems in multi_poem_clusters.items():
    sources = {poem_sources.get(p, 'unknown') for p in poems}
    if len(sources) > 1:
        multi_source_clusters[cid] = poems

for cluster_id, poems in multi_source_clusters.items():
    print(f"\n=== Cluster {cluster_id} ({len(poems)} poems) ===")
    for poem_id in poems:
        verses = poem_texts.get(poem_id, [])
        source = poem_sources.get(poem_id, 'unknown')
        print(f"\nPoem ID: {poem_id} | Source: {source}")
        verses_str = [str(v) for v in verses if pd.notna(v)]  # optional: skip NaN
        print("\n".join(verses_str))
        print("-" * 40)
